In [46]:
import numpy as np
import pandas as pd
from bs4  import BeautifulSoup
import requests
import time
from datetime import datetime 
import warnings
warnings.filterwarnings("ignore")
import sys
import json
from json import loads, dumps
import lxml
from requests import ConnectionError, ReadTimeout, ConnectTimeout, HTTPError, Timeout
import xml
import re
from natsort import natsorted
import xml.etree.ElementTree as ET
import xmltodict
from xml.parsers.expat import ExpatError
ewc = ['SHOT', 'HIT', 'BLOCK', 'MISS', 'GIVE', 'TAKE', 'GOAL']

# SCRAPE API SCHEDULE

In [47]:
def scrape_schedule(start_date, end_date):
    url = 'https://statsapi.web.nhl.com/api/v1/schedule?startDate=' + start_date + '&endDate=' + end_date
    page = requests.get(url, timeout = 500)
    loaddict = json.loads(page.content)
    date_list = (loaddict['dates'])
    date_df = pd.DataFrame(date_list)
    
    gamedf = pd.DataFrame()

    for i in range (0, len(date_df)):
        datedf = pd.DataFrame(date_df.games.iloc[i])
        gamedf = gamedf.append(datedf)
    global team_df
    team_df = pd.DataFrame(gamedf['teams'].values.tolist(), index = gamedf.index)
    away_df = pd.DataFrame(team_df['away'].values.tolist(), index = team_df.index)
    home_df = pd.DataFrame(team_df['home'].values.tolist(), index = team_df.index)
    away_team_df = pd.DataFrame(away_df['team'].values.tolist(), index = away_df.index)
    home_team_df = pd.DataFrame(home_df['team'].values.tolist(), index = home_df.index)

    gamedf = gamedf.assign(
        state = pd.DataFrame(gamedf['status'].values.tolist(), index = gamedf.index)['detailedState'],
        homename = home_team_df['name'],
        homeid = home_team_df['id'],
        homescore = home_df['score'],
        awayname = away_team_df['name'],
        awayid = away_team_df['id'],
        awayscore = away_df['score'],
        venue = pd.DataFrame(gamedf['venue'].values.tolist(), index = gamedf.index)['name'],
        gameDate = pd.to_datetime(gamedf['gameDate']).dt.tz_convert('EST')
    )

    gamedf = gamedf.loc[:, ['gamePk', 'link', 'gameType', 'season', 'gameDate','homeid', 'homename',  'homescore','awayid', 'awayname',  'awayscore', 'state', 'venue']].rename(
        columns = {'gamePk':'ID', 'gameType':'type', 'gameDate':'date'})

    return gamedf

# Harry's Strip HTML Function

In [48]:
def hs_strip_html(td):
    """
    Function from Harry Shomer's Github
    
    Strip html tags and such 
    
    :param td: pbp
    
    :return: list of plays (which contain a list of info) stripped of html
    """
    for y in range(len(td)):
        # Get the 'br' tag for the time column...this get's us time remaining instead of elapsed and remaining combined
        if y == 3:
            td[y] = td[y].get_text()   # This gets us elapsed and remaining combined-< 3:0017:00
            index = td[y].find(':')
            td[y] = td[y][:index+3]
        elif (y == 6 or y == 7) and td[0] != '#':
            # 6 & 7-> These are the player 1 ice one's
            # The second statement controls for when it's just a header
            baz = td[y].find_all('td')
            bar = [baz[z] for z in range(len(baz)) if z % 4 != 0]  # Because of previous step we get repeats...delete some

            # The setup in the list is now: Name/Number->Position->Blank...and repeat
            # Now strip all the html
            players = []
            for i in range(len(bar)):
                if i % 3 == 0:
                    try:
                        name = return_name_html(bar[i].find('font')['title'])
                        number = bar[i].get_text().strip('\n')  # Get number and strip leading/trailing newlines
                    except KeyError:
                        name = ''
                        number = ''
                elif i % 3 == 1:
                    if name != '':
                        position = bar[i].get_text()
                        players.append([name, number, position])

            td[y] = players
        else:
            td[y] = td[y].get_text()

    return td

def group_if_not_none(result):
    if result is not None:
        result = result.group()
    return result

# Scrape HTML Roster

In [49]:
def scrape_html_roster(season, game_id):
    url = 'http://www.nhl.com/scores/htmlreports/' + season + '/RO0' + game_id + '.HTM'
    page = requests.get(url)
    soup = BeautifulSoup(page.content.decode('ISO-8859-1'), 'lxml', multi_valued_attributes = None)
    
    teamsoup = soup.find_all('td', {'align':'center', 'class':['teamHeading + border', 'teamHeading + border '], 'width':'50%'})
    away_team = teamsoup[0].get_text()
    home_team = teamsoup[1].get_text()
    
    home_player_soup = (soup.find_all('table', {'align':'center', 'border':'0', 'cellpadding':'0', 
                        'cellspacing':'0', 'width':'100%', 'xmlns:ext':''}))[1].find_all('td')

    length = int(len(home_player_soup)/3)

    home_player_df = pd.DataFrame(np.array(home_player_soup).reshape(length, 3))

    home_player_df.columns = home_player_df.iloc[0]

    home_player_df = home_player_df.drop(0).assign(team = 'home', team_name = home_team)

    away_player_soup = (soup.find_all('table', {'align':'center', 'border':'0', 'cellpadding':'0', 
                            'cellspacing':'0', 'width':'100%', 'xmlns:ext':''}))[0].find_all('td')

    length = int(len(away_player_soup)/3)

    away_player_df = pd.DataFrame(np.array(away_player_soup).reshape(length, 3))

    away_player_df.columns = away_player_df.iloc[0]

    away_player_df = away_player_df.drop(0).assign(team = 'away', team_name = away_team)
    
    #global home_scratch_soup
    
    if len(soup.find_all('table', {'align':'center', 'border':'0', 'cellpadding':'0', 
                            'cellspacing':'0', 'width':'100%', 'xmlns:ext':''}))>3:

        home_scratch_soup = (soup.find_all('table', {'align':'center', 'border':'0', 'cellpadding':'0', 
                                'cellspacing':'0', 'width':'100%', 'xmlns:ext':''}))[3].find_all('td')
    
        if len(home_scratch_soup)>1:

            length = int(len(home_scratch_soup)/3)

            home_scratch_df = pd.DataFrame(np.array(home_scratch_soup).reshape(length, 3))

            home_scratch_df.columns = home_scratch_df.iloc[0]

            home_scratch_df = home_scratch_df.drop(0).assign(team = 'home', team_name = home_team)

    if 'home_scratch_df' not in locals():
        
        home_scratch_df = pd.DataFrame()
        
    if len(soup.find_all('table', {'align':'center', 'border':'0', 'cellpadding':'0', 
                            'cellspacing':'0', 'width':'100%', 'xmlns:ext':''}))>2:
    
        away_scratch_soup = (soup.find_all('table', {'align':'center', 'border':'0', 'cellpadding':'0', 
                                'cellspacing':'0', 'width':'100%', 'xmlns:ext':''}))[2].find_all('td')
        
        if len(away_scratch_soup)>1:

            length = int(len(away_scratch_soup)/3)

            away_scratch_df = pd.DataFrame(np.array(away_scratch_soup).reshape(length, 3))

            away_scratch_df.columns = away_scratch_df.iloc[0]

            away_scratch_df = away_scratch_df.drop(0).assign(team = 'away', team_name = away_team)
        
    if 'away_scratch_df' not in locals():
        
        away_scratch_df = pd.DataFrame()

    player_df = pd.concat([home_player_df, away_player_df]).assign(status = 'player')
    scratch_df = pd.concat([home_scratch_df, away_scratch_df]).assign(status = 'scratch')
    roster_df = pd.concat([player_df, scratch_df])
    
    roster_df = roster_df.assign(team = np.where(roster_df.team=='CANADIENS MONTREAL', 'MONTREAL CANADIENS', roster_df.team))
    
    # FIX NAMES

    roster_df = roster_df.rename(columns = {'Nom/Name':'Name'})
    
    roster_df.Name = roster_df.Name.str.split('(').str[0].str.strip()
    
    # Max Pacioretty doesn't exist in ESPN in 2009-2010, sadly.
    
    roster_df['Name'] = np.where(roster_df['Name'].str.contains('ALEXANDRE '), 
                                roster_df.Name.str.replace('ALEXANDRE ', 'ALEX '),
                                roster_df['Name'])
    
    roster_df['Name'] = np.where(roster_df['Name'].str.contains('ALEXANDER '), 
                                roster_df.Name.str.replace('ALEXANDER ', 'ALEX '),
                                roster_df['Name'])
    
    roster_df['Name'] = np.where(roster_df['Name'].str.contains('CHRISTOPHER '), 
                                roster_df.Name.str.replace('CHRISTOPHER ', 'CHRIS '),
                                roster_df['Name'])
    
    roster_df = roster_df.assign(Name = 
    (np.where(roster_df['Name']== "ANDREI KASTSITSYN" , "ANDREI KOSTITSYN",
    (np.where(roster_df['Name']== "AJ GREER" , "A.J.  GREER",
    (np.where(roster_df['Name']== "ANDREW GREENE" , "ANDY GREENE",
    (np.where(roster_df['Name']== "ANDREW WOZNIEWSKI" , "ANDY WOZNIEWSKI", 
    (np.where(roster_df['Name']== "ANTHONY DEANGELO" , "TONY DEANGELO",
    (np.where(roster_df['Name']== "BATES (JON) BATTAGLIA" , "BATES BATTAGLIA",
    (np.where(roster_df['Name'].isin(["BJ CROMBEEN", "B.J. CROMBEEN", "BRANDON CROMBEEN", "B J CROMBEEN"]) , "B.J. CROMBEEN", 
    (np.where(roster_df['Name']== "BRADLEY MILLS" , "BRAD MILLS",
    (np.where(roster_df['Name']== "CAMERON BARKER" , "CAM BARKER", 
    (np.where(roster_df['Name']== "COLIN (JOHN) WHITE" , "COLIN WHITE",
    (np.where(roster_df['Name']== "CRISTOVAL NIEVES" , "BOO NIEVES",
    (np.where(roster_df['Name']== "CHRIS VANDE VELDE" , "CHRIS VANDEVELDE", 
    (np.where(roster_df['Name']== "DANNY BRIERE" , "DANIEL BRIERE",
    (np.where(roster_df['Name'].isin(["DAN CLEARY", "DANNY CLEARY"]) , "DANIEL CLEARY",
    (np.where(roster_df['Name']== "DANIEL GIRARDI" , "DAN GIRARDI", 
    (np.where(roster_df['Name']== "DANNY O'REGAN" , "DANIEL O'REGAN",
    (np.where(roster_df['Name']== "DANIEL CARCILLO" , "DAN CARCILLO", 
    (np.where(roster_df['Name']== "DAVID JOHNNY ODUYA" , "JOHNNY ODUYA", 
    (np.where(roster_df['Name']== "DAVID BOLLAND" , "DAVE BOLLAND", 
    (np.where(roster_df['Name']== "DENIS JR  GAUTHIER" , "DENIS GAUTHIER",
    (np.where(roster_df['Name']== "DWAYNE KING" , "DJ KING", 
    (np.where(roster_df['Name']== "EDWARD PURCELL" , "TEDDY PURCELL", 
    (np.where(roster_df['Name']== "EMMANUEL FERNANDEZ" , "MANNY FERNANDEZ", 
    (np.where(roster_df['Name']== "EMMANUEL LEGACE" , "MANNY LEGACE", 
    (np.where(roster_df['Name']== "EVGENII DADONOV" , "EVGENY DADONOV", 
    (np.where(roster_df['Name']== "FREDDY MODIN" , "FREDRIK MODIN", 
    (np.where(roster_df['Name']== "FREDERICK MEYER IV" , "FREDDY MEYER",
    (np.where(roster_df['Name']== "HARRISON ZOLNIERCZYK" , "HARRY ZOLNIERCZYK", 
    (np.where(roster_df['Name']== "ILJA BRYZGALOV" , "ILYA BRYZGALOV", 
    (np.where(roster_df['Name']== "JACOB DOWELL" , "JAKE DOWELL",
    (np.where(roster_df['Name']== "JAMES HOWARD" , "JIMMY HOWARD", 
    (np.where(roster_df['Name']== "JAMES VANDERMEER" , "JIM VANDERMEER",
    (np.where(roster_df['Name']== "JAMES WYMAN" , "JT WYMAN",
    (np.where(roster_df['Name']== "JOHN HILLEN III" , "JACK HILLEN",
    (np.where(roster_df['Name']== "JOHN ODUYA" , "JOHNNY ODUYA",
    (np.where(roster_df['Name']== "JOHN PEVERLEY" , "RICH PEVERLEY",
    (np.where(roster_df['Name']== "JONATHAN SIM" , "JON SIM",
    (np.where(roster_df['Name']== "JONATHON KALINSKI" , "JON KALINSKI",
    (np.where(roster_df['Name']== "JONATHAN AUDY-MARCHESSAULT" , "JONATHAN MARCHESSAULT", 
    (np.where(roster_df['Name']== "JOSEPH CRABB" , "JOEY CRABB",
    (np.where(roster_df['Name']== "JOSEPH CORVO" , "JOE CORVO", 
    (np.where(roster_df['Name']== "JOSHUA BAILEY" , "JOSH BAILEY",
    (np.where(roster_df['Name']== "JOSHUA HENNESSY" , "JOSH HENNESSY", 
    (np.where(roster_df['Name']== "JOSHUA MORRISSEY" , "JOSH MORRISSEY",
    (np.where(roster_df['Name']== "JEAN-FRANCOIS JACQUES" , "J-F JACQUES", 
    (np.where(roster_df['Name'].isin(["J P DUMONT", "JEAN-PIERRE DUMONT"]) , "J-P DUMONT", 
    (np.where(roster_df['Name']== "JT COMPHER" , "J.T. COMPHER",
    (np.where(roster_df['Name']== "KRISTOPHER LETANG" , "KRIS LETANG", 
    (np.where(roster_df['Name']== "KRYSTOFER BARCH" , "KRYS BARCH", 
    (np.where(roster_df['Name']== "KRYSTOFER KOLANOS" , "KRYS KOLANOS",
    (np.where(roster_df['Name']== "MARC POULIOT" , "MARC-ANTOINE POULIOT",
    (np.where(roster_df['Name']== "MARTIN ST LOUIS" , "MARTIN ST. LOUIS", 
    (np.where(roster_df['Name']== "MARTIN ST PIERRE" , "MARTIN ST. PIERRE",
    (np.where(roster_df['Name']== "MARTY HAVLAT" , "MARTIN HAVLAT",
    (np.where(roster_df['Name']== "MATTHEW CARLE" , "MATT CARLE", 
    (np.where(roster_df['Name']== "MATHEW DUMBA" , "MATT DUMBA",
    (np.where(roster_df['Name']== "MATTHEW BENNING" , "MATT BENNING", 
    (np.where(roster_df['Name']== "MATTHEW IRWIN" , "MATT IRWIN",
    (np.where(roster_df['Name']== "MATTHEW NIETO" , "MATT NIETO",
    (np.where(roster_df['Name']== "MATTHEW STAJAN" , "MATT STAJAN",
    (np.where(roster_df['Name']== "MAXIM MAYOROV" , "MAKSIM MAYOROV",
    (np.where(roster_df['Name']== "MAXIME TALBOT" , "MAX TALBOT", 
    (np.where(roster_df['Name']== "MAXWELL REINHART" , "MAX REINHART",
    (np.where(roster_df['Name']== "MICHAEL BLUNDEN" , "MIKE BLUNDEN",
    (np.where(roster_df['Name']== "MICHAËL BOURNIVAL" , "MICHAEL BOURNIVAL",
    (np.where(roster_df['Name']== "MICHAEL CAMMALLERI" , "MIKE CAMMALLERI", 
    (np.where(roster_df['Name']== "MICHAEL FERLAND" , "MICHEAL FERLAND", 
    (np.where(roster_df['Name']== "MICHAEL GRIER" , "MIKE GRIER",
    (np.where(roster_df['Name']== "MICHAEL KNUBLE" , "MIKE KNUBLE",
    (np.where(roster_df['Name']== "MICHAEL KOMISAREK" , "MIKE KOMISAREK",
    (np.where(roster_df['Name']== "MICHAEL MATHESON" , "MIKE MATHESON",
    (np.where(roster_df['Name']== "MICHAEL MODANO" , "MIKE MODANO",
    (np.where(roster_df['Name']== "MICHAEL RUPP" , "MIKE RUPP",
    (np.where(roster_df['Name']== "MICHAEL SANTORELLI" , "MIKE SANTORELLI", 
    (np.where(roster_df['Name']== "MICHAEL SILLINGER" , "MIKE SILLINGER",
    (np.where(roster_df['Name']== "MITCHELL MARNER" , "MITCH MARNER", 
    (np.where(roster_df['Name']== "NATHAN GUENIN" , "NATE GUENIN",
    (np.where(roster_df['Name']== "NICHOLAS BOYNTON" , "NICK BOYNTON",
    (np.where(roster_df['Name']== "NICHOLAS DRAZENOVIC" , "NICK DRAZENOVIC", 
    (np.where(roster_df['Name']== "NICKLAS BERGFORS" , "NICLAS BERGFORS",
    (np.where(roster_df['Name']== "NICKLAS GROSSMAN" , "NICKLAS GROSSMANN", 
    (np.where(roster_df['Name']== "NICOLAS PETAN" , "NIC PETAN", 
    (np.where(roster_df['Name']== "NIKLAS KRONVALL" , "NIKLAS KRONWALL",
    (np.where(roster_df['Name']== "NIKOLAI ANTROPOV" , "NIK ANTROPOV",
    (np.where(roster_df['Name']== "NIKOLAI KULEMIN" , "NIKOLAY KULEMIN", 
    (np.where(roster_df['Name']== "NIKOLAI ZHERDEV" , "NIKOLAY ZHERDEV",
    (np.where(roster_df['Name']== "OLIVIER MAGNAN-GRENIER" , "OLIVIER MAGNAN",
    (np.where(roster_df['Name']== "PAT MAROON" , "PATRICK MAROON", 
    (np.where(roster_df['Name'].isin(["P. J. AXELSSON", "PER JOHAN AXELSSON"]) , "P.J. AXELSSON",
    (np.where(roster_df['Name'].isin(["PK SUBBAN", "P.K SUBBAN"]) , "P.K.  SUBBAN", 
    (np.where(roster_df['Name'].isin(["PIERRE PARENTEAU", "PIERRE-ALEX PARENTEAU", "PIERRE-ALEXANDRE PARENTEAU", "PA PARENTEAU", "P.A PARENTEAU", "P-A PARENTEAU"]) , "P A  PARENTEAU", 
    (np.where(roster_df['Name']== "PHILIP VARONE" , "PHIL VARONE",
    (np.where(roster_df['Name']== "QUINTIN HUGHES" , "QUINN HUGHES",
    (np.where(roster_df['Name']== "RAYMOND MACIAS" , "RAY MACIAS",
    (np.where(roster_df['Name']== "RJ UMBERGER" , "R.J. UMBERGER",
    (np.where(roster_df['Name']== "ROBERT BLAKE" , "ROB BLAKE",
    (np.where(roster_df['Name']== "ROBERT EARL" , "ROBBIE EARL",
    (np.where(roster_df['Name']== "ROBERT HOLIK" , "BOBBY HOLIK",
    (np.where(roster_df['Name']== "ROBERT SCUDERI" , "ROB SCUDERI",
    roster_df['Name']))))))))))))))))))))))))))))))))))))))))))))))))))))))
    )))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))
    ))))))))))
    
    roster_df['Name'] = (np.where(roster_df['Name']== "RODNEY PELLEY" , "ROD PELLEY",
    (np.where(roster_df['Name']== "SIARHEI KASTSITSYN" , "SERGEI KOSTITSYN",
    (np.where(roster_df['Name']== "SIMEON VARLAMOV" , "SEMYON VARLAMOV", 
    (np.where(roster_df['Name']== "STAFFAN KRONVALL" , "STAFFAN KRONWALL",
    (np.where(roster_df['Name']== "STEVEN REINPRECHT" , "STEVE REINPRECHT",
    (np.where(roster_df['Name']== "TJ GALIARDI" , "T.J. GALIARDI",
    (np.where(roster_df['Name']== "TJ HENSICK" , "T.J  HENSICK",
    (np.where(roster_df['Name'].isin(["TJ OSHIE", "T.J OSHIE"]) , "T.J. OSHIE", 
    (np.where(roster_df['Name']== "TOBY ENSTROM" , "TOBIAS ENSTROM", 
    (np.where(roster_df['Name']== "TOMMY SESTITO" , "TOM SESTITO",
    (np.where(roster_df['Name']== "VACLAV PROSPAL" , "VINNY PROSPAL",
    (np.where(roster_df['Name']== "VINCENT HINOSTROZA" , "VINNIE HINOSTROZA",
    (np.where(roster_df['Name']== "WILLIAM THOMAS" , "BILL THOMAS",
    (np.where(roster_df['Name']== "ZACHARY ASTON-REESE" , "ZACH ASTON-REESE",
    (np.where(roster_df['Name']== "ZACHARY SANFORD" , "ZACH SANFORD",
    (np.where(roster_df['Name']== "ZACHERY STORTINI" , "ZACK STORTINI",
    (np.where(roster_df['Name']== "MATTHEW MURRAY" , "MATT MURRAY",
    (np.where(roster_df['Name']== "J-SEBASTIEN AUBIN" , "JEAN-SEBASTIEN AUBIN",
    (np.where(roster_df['Name'].isin(["J.F.  BERUBE", "JEAN-FRANCOIS BERUBE"]) , "J-F BERUBE", 
    (np.where(roster_df['Name']== "JEFF DROUIN-DESLAURIERS" , "JEFF DESLAURIERS", 
    (np.where(roster_df['Name']== "NICHOLAS BAPTISTE" , "NICK BAPTISTE",
    (np.where(roster_df['Name']== "OLAF KOLZIG" , "OLIE KOLZIG",
    (np.where(roster_df['Name']== "STEPHEN VALIQUETTE" , "STEVE VALIQUETTE",
    (np.where(roster_df['Name']== "THOMAS MCCOLLUM" , "TOM MCCOLLUM",
    (np.where(roster_df['Name']== "TIMOTHY JR  THOMAS" , "TIM THOMAS",
    (np.where(roster_df['Name']== "TIM GETTINGER" , "TIMOTHY GETTINGER",
    (np.where(roster_df['Name']== "NICHOLAS SHORE" , "NICK SHORE",
    (np.where(roster_df['Name']== "T.J  TYNAN" , "TJ TYNAN",
    (np.where(roster_df['Name']== "ALEXIS LAFRENI?RE" , "ALEXIS LAFRENIÈRE",
    (np.where(roster_df['Name']== "ALEXIS LAFRENIERE" , "ALEXIS LAFRENIÈRE", 
    (np.where(roster_df['Name']== "ALEXIS LAFRENIÃRE" , "ALEXIS LAFRENIÈRE",
    (np.where(roster_df['Name']== "TIM STUTZLE" , "TIM STÜTZLE",
    (np.where(roster_df['Name']== "TIM ST?TZLE" , "TIM STÜTZLE",
    (np.where(roster_df['Name']== "TIM STÃTZLE" , "TIM STÜTZLE",
    (np.where(roster_df['Name']== "EGOR SHARANGOVICH" , "YEGOR SHARANGOVICH",
    (np.where(roster_df['Name']== "CALLAN FOOTE" , "CAL FOOTE",
    (np.where(roster_df['Name']== "MATTIAS JANMARK-NYLEN" , "MATTIAS JANMARK",
    (np.where(roster_df['Name']== "JOSH DUNNE" , "JOSHUA DUNNE",roster_df['Name'])))))))))))))))))))))))))))))))))))))))))))
    )))))))))))))))))))))))))))))))))

    return roster_df

# SCRAPE HTML SHIFTS

In [50]:
def scrape_html_shifts(season, game_id):
    
    url = 'http://www.nhl.com/scores/htmlreports/' + season + '/TH0' + game_id + '.HTM'
    page = (requests.get(url))
    soup = BeautifulSoup(page.content.decode('ISO-8859-1'), 'lxml', multi_valued_attributes = None)
    found = soup.find_all('td', {'class':['playerHeading + border', 'lborder + bborder']})
    if len(found)==0:
        raise IndexError('This game has no shift data.')
    thisteam = soup.find('td', {'align':'center', 'class':'teamHeading + border'}).get_text()

    players = dict()

    for i in range(len(found)):
        line = found[i].get_text()
        if ', ' in line:
            name = line.split(',')
            number = name[0].split(' ')[0].strip()
            last_name =  name[0].split(' ')[1].strip()
            first_name = name[1].strip()
            full_name = first_name + " " + last_name
            players[full_name] = dict()
            players[full_name]['number'] = number
            players[full_name]['name'] = full_name
            players[full_name]['shifts'] = []
        else:
            players[full_name]['shifts'].extend([line])

    alldf = pd.DataFrame()

    for key in players.keys(): 
        length = int(len(np.array((players[key]['shifts'])))/5)
        df = pd.DataFrame(np.array((players[key]['shifts'])).reshape(length, 5)).rename(
        columns = {0:'shift_number', 1:'period', 2:'shift_start', 3:'shift_end', 4:'duration'})
        df = df.assign(name = players[key]['name'],
                      number = players[key]['number'],
                      team = thisteam,
                      venue = "home")
        alldf = alldf.append(df)
        
    home_shifts = alldf
    
    url = 'http://www.nhl.com/scores/htmlreports/' + season + '/TV0' + game_id + '.HTM'
    page = (requests.get(url))
    soup = BeautifulSoup(page.content.decode('ISO-8859-1'), 'lxml', multi_valued_attributes = None)
    found = soup.find_all('td', {'class':['playerHeading + border', 'lborder + bborder']})
    thisteam = soup.find('td', {'align':'center', 'class':'teamHeading + border'}).get_text()

    players = dict()

    for i in range(len(found)):
        line = found[i].get_text()
        if ', ' in line:
            name = line.split(',')
            number = name[0].split(' ')[0].strip()
            last_name =  name[0].split(' ')[1].strip()
            first_name = name[1].strip()
            full_name = first_name + " " + last_name
            players[full_name] = dict()
            players[full_name]['number'] = number
            players[full_name]['name'] = full_name
            players[full_name]['shifts'] = []
        else:
            players[full_name]['shifts'].extend([line])

    alldf = pd.DataFrame()

    for key in players.keys(): 
        length = int(len(np.array((players[key]['shifts'])))/5)
        df = pd.DataFrame(np.array((players[key]['shifts'])).reshape(length, 5)).rename(
        columns = {0:'shift_number', 1:'period', 2:'shift_start', 3:'shift_end', 4:'duration'})
        df = df.assign(name = players[key]['name'],
                      number = players[key]['number'],
                      team = thisteam,
                      venue = "away")
        alldf = alldf.append(df)

    away_shifts = alldf
    
    global all_shifts
    
    all_shifts = pd.concat([home_shifts, away_shifts])
    
    all_shifts = all_shifts.assign(start_time = all_shifts.shift_start.str.split('/').str[0])
    
    all_shifts = all_shifts.assign(end_time = all_shifts.shift_end.str.split('/').str[0])
    
    #all_shifts = all_shifts[~all_shifts.end_time.str.contains('\xa0')]
    
    all_shifts.period = (np.where(all_shifts.period=='OT', 4, all_shifts.period)).astype(int)
    
    all_shifts = all_shifts.assign(end_time = np.where(~all_shifts.shift_end.str.contains('\xa0'), all_shifts.end_time,
              (np.where(
              (((pd.to_datetime(((60 * (all_shifts.start_time.str.split(':').str[0].astype(int))) + 
              (all_shifts.start_time.str.split(':').str[1].astype(int)) + 
                (60 * (all_shifts.duration.str.split(':').str[0].astype(int))).astype(int) +
              (all_shifts.duration.str.split(':').str[1].astype(int))).astype(int), unit = 's'))).dt.time).astype(str).str[3:].str[0]=='0',
              (((pd.to_datetime(((60 * (all_shifts.start_time.str.split(':').str[0].astype(int))) + 
              (all_shifts.start_time.str.split(':').str[1].astype(int)) + 
                (60 * (all_shifts.duration.str.split(':').str[0].astype(int))).astype(int) +
              (all_shifts.duration.str.split(':').str[1].astype(int))).astype(int), unit = 's'))).dt.time).astype(str).str[4:],
              (((pd.to_datetime(((60 * (all_shifts.start_time.str.split(':').str[0].astype(int))) + 
              (all_shifts.start_time.str.split(':').str[1].astype(int)) + 
                (60 * (all_shifts.duration.str.split(':').str[0].astype(int))).astype(int) +
              (all_shifts.duration.str.split(':').str[1].astype(int))).astype(int), unit = 's'))).dt.time).astype(str).str[4:]))))
    
    myshifts = all_shifts
    
    myshifts.start_time = myshifts.start_time.str.strip()
    myshifts.end_time = myshifts.end_time.str.strip()
    
    changes_on = myshifts.groupby(['team', 'period', 'start_time']).agg(
        on = ('name', ', '.join),
        on_numbers = ('number', ', '.join),
        number_on = ('name', 'count')
    ).reset_index().rename(columns = {'start_time':'time'}).sort_values(by = ['team', 'period', 'time'])
    
    changes_off = myshifts.groupby(['team', 'period', 'end_time']).agg(
        off = ('name', ', '.join),
        off_numbers = ('number', ', '.join),
        number_off = ('name', 'count')
    ).reset_index().rename(columns = {'end_time':'time'}).sort_values(by = ['team', 'period', 'time'])
    
    all_on = changes_on.merge(changes_off, on = ['team', 'period', 'time'], how = 'left')
    off_only = changes_off.merge(changes_on, on = ['team', 'period', 'time'], how = 'left', indicator = True)[
    changes_off.merge(changes_on, on = ['team', 'period', 'time'], how = 'left', indicator = True)['_merge']!='both']
    full_changes = pd.concat([all_on, off_only]).sort_values(by = ['period', 'time']).drop(columns = ['_merge'])
    
    full_changes['period_seconds'] = full_changes.time.str.split(':').str[0].astype(int) * 60 + full_changes.time.str.split(':').str[1].astype(int)

    full_changes['game_seconds'] = (np.where(full_changes.period<5, 
                                   (((full_changes.period - 1) * 1200) + full_changes.period_seconds),
                          3900))
    
    full_changes = full_changes.assign(team = np.where(full_changes.team=='CANADIENS MONTREAL', 'MONTREAL CANADIENS', full_changes.team))
        
    return full_changes.reset_index(drop = True)#.drop(columns = ['time', 'period_seconds'])

# Scrape API Events

In [73]:
def scrape_api_events(game_id, drop_description = True, shift_to_espn = False):
    
    if shift_to_espn == True:
        raise KeyError
    
    page = requests.get(str('https://statsapi.web.nhl.com/api/v1/game/' + str(game_id) + '/feed/live'))
    
    if str(page) == '<Response [404]>':
        raise KeyError('You got the 404 error; game data could not be found.')
    
    loaddict = json.loads(page.content)
    
    if loaddict['liveData']['plays']['allPlays'] != []:
    
        eventdf = pd.DataFrame(loaddict['liveData']['plays']['allPlays'])

        coordsdf = pd.DataFrame(eventdf['coordinates'].values.tolist(), index = eventdf.index)
        resultdf = pd.DataFrame(eventdf['result'].values.tolist(), index = eventdf.index)
        aboutdf = pd.DataFrame(eventdf['about'].values.tolist(), index = eventdf.index)
        scoredf = pd.DataFrame(aboutdf['goals'].values.tolist(), index = aboutdf.index)
        playerdf = pd.DataFrame(eventdf['players'])
        teamdf = eventdf['team'].apply(pd.Series)
        clean = playerdf[~pd.isna(playerdf.players)].reset_index()
        clean_index = clean.loc[:, ['index']]
        player1 = pd.DataFrame((pd.DataFrame(clean.reset_index()['players'].values.tolist())[0].values.tolist()))
        player1df = pd.concat([clean_index, pd.DataFrame(player1['player'].values.tolist())], axis = 1).assign(playerType = player1['playerType']).rename(
            columns = {'id':'player1id', 'fullName':'player1name', 'link':'player1link', 'playerType':'player1type'})
        player2 = pd.concat([clean_index, pd.DataFrame((pd.DataFrame(clean['players'].values.tolist())[1]))], axis = 1)
        player2 = player2[player2[1].notnull()]
        player2df = pd.concat([player2.reset_index(drop = True), 
            (pd.DataFrame(pd.DataFrame(player2[1].values.tolist())['player'].values.tolist()).assign(playerType = (pd.DataFrame(player2[1].values.tolist())).loc[:, ['playerType']]))], axis = 1).drop(
        columns = 1).rename(
            columns = {'id':'player2id', 'fullName':'player2name', 'link':'player2link', 'playerType':'player2type'})

        if len((pd.DataFrame(clean['players'].values.tolist())).columns) > 2:

            player3 = pd.concat([clean_index, pd.DataFrame((pd.DataFrame(clean['players'].values.tolist())[2]))], axis = 1)
            player3 = player3[player3[2].notnull()]
            player3df = pd.concat([player3.reset_index(drop = True), 
                (pd.DataFrame(pd.DataFrame(player3[2].values.tolist())['player'].values.tolist()).assign(playerType = (pd.DataFrame(player3[2].values.tolist())).loc[:, ['playerType']]))], axis = 1).drop(
            columns = 2).rename(
                columns = {'id':'player3id', 'fullName':'player3name', 'link':'player3link', 'playerType':'player3type'})
        else: 
            player3df = pd.DataFrame(columns = ['index', 'player3id', 'player3name', 'player3link', 'player3type'])

        if len((pd.DataFrame(clean['players'].values.tolist())).columns) > 3:  

            player4 = pd.concat([clean_index, pd.DataFrame((pd.DataFrame(clean['players'].values.tolist())[3]))], axis = 1)
            player4 = player4[player4[3].notnull()]
            player4df = pd.concat([player4.reset_index(drop = True), 
                (pd.DataFrame(pd.DataFrame(player4[3].values.tolist())['player'].values.tolist()).assign(playerType = (pd.DataFrame(player4[3].values.tolist())).loc[:, ['playerType']]))], axis = 1).drop(
            columns = 3).rename(
                columns = {'id':'player4id', 'fullName':'player4name', 'link':'player4link', 'playerType':'player4type'})
        else: 
            player4df = pd.DataFrame(columns = ['index', 'player4id', 'player4name', 'player4link', 'player4type'])

        finaldf = eventdf.assign(
            hometeam = loaddict['gameData']['teams']['home']['triCode'],
            hometeamfull = loaddict['gameData']['teams']['home']['name'],
            awayteam = loaddict['gameData']['teams']['away']['triCode'],
            awayteamfull = loaddict['gameData']['teams']['away']['name'],
            description = resultdf['description'],
            event = resultdf['eventTypeId'],
            detail = resultdf['secondaryType'],
            coords_x = coordsdf['x'],
            coords_y = coordsdf['y'],
            period = aboutdf['period'],
            time = aboutdf['periodTime'],
            homescore = scoredf['home'],
            awayscore = scoredf['away'],
            eventteam = teamdf['triCode'],
            eventteamfull = teamdf['name'],
            eventidx = aboutdf['eventIdx'],
            eventNumber = aboutdf['eventId'],
            session = loaddict['gameData']['game']['type'])

        finaldf = finaldf.drop(columns = ['result', 'about', 'coordinates', 'players', 'team'])

        finaldf = finaldf.reset_index().merge(
        player1df, on = 'index', how = 'left').merge(
        player2df, on = 'index', how = 'left').merge(
        player3df, on = 'index', how = 'left').merge(
        player4df, on = 'index', how = 'left')
        
        finaldf = finaldf.assign(
            awayteamfull = finaldf.awayteamfull.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'),
            hometeamfull = finaldf.hometeamfull.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'),
            eventteamfull = finaldf.eventteamfull.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
        
        finaldf = finaldf.assign(
            player1name = np.where((finaldf.player1name=='Sebastian Aho') & (finaldf.eventteam=='NYI'),
                     'Sebastian Aho (SWE)',
                     finaldf.player1name
                    ))

        api_events = finaldf
    
        api_events.period = api_events.period.astype(int)
        api_events.time = api_events.time.astype(str)

        api_events.event = np.where(api_events.event=='BLOCKED_SHOT', 'BLOCK',
        np.where(api_events.event=='BLOCKEDSHOT', 'BLOCK',
                np.where(api_events.event=='MISSED_SHOT', 'MISS',
                        np.where(api_events.event=='FACEOFF', 'FAC',
                                np.where(api_events.event=='PENALTY', 'PENL',
                                        np.where(api_events.event=='GIVEAWAY', 'GIVE',
                                                np.where(api_events.event=='TAKEAWAY', 'TAKE',
                                                         np.where(api_events.event=='MISSEDSHOT', 'MISS',
                                                                  api_events.event))))))))

        api_events = api_events[api_events.event.isin(['TAKE', 'GIVE', 'MISS', 'HIT', 'SHOT', 'BLOCK', 'GOAL', 'PENL', 'FAC'])]

        api_events['awayteamfull'] = (api_events.awayteamfull.str.upper())
        api_events['hometeamfull'] = (api_events.hometeamfull.str.upper())
        api_events['eventteamfull'] = (api_events.eventteamfull.str.upper())

        api_events['period_seconds'] = api_events.time.str.split(':').str[0].astype(int) * 60 + api_events.time.str.split(':').str[1].astype(int)

        api_events['game_seconds'] = (np.where(api_events.period<5, 
                                       (((api_events.period - 1) * 1200) + api_events.period_seconds),
                              3900))


        api_events = api_events.loc[:, ['period_seconds', 'game_seconds', 'event', 'session', 'coords_x', 'coords_y', 'description', 'period',
                                        'eventteam', 'eventteamfull', 'hometeamfull', 'awayteamfull', 'player1name', 'player2name', 'player3name', 'player4name']].rename(
            columns = {'eventteamfull':'event_team'})

        api_events = api_events.assign(
        player1name = api_events.player1name.str.upper(),
        player2name = api_events.player2name.str.upper(),
        player3name = api_events.player3name.str.upper()
        ).drop(columns = 'player4name').rename(columns = {'player1name':'ep1_name', 'player2name':'ep2_name', 'player3name':'ep3_name'})
    
        api_events = api_events.assign(event_team = np.where(api_events.event!='BLOCK', api_events.event_team,
            np.where(api_events.event_team==api_events.hometeamfull, api_events.awayteamfull, api_events.hometeamfull)))
        
        api_events = api_events.assign(ep1_name = np.where(api_events.event!='BLOCK', api_events.ep1_name, api_events.ep2_name))
    
        api_events = api_events.sort_values(by = ['game_seconds', 'event_team', 'ep1_name'])
    
        api_events = api_events.assign(version = 
                               (np.where(
                               (api_events.event==api_events.event.shift()) & 
                               (api_events.ep1_name==api_events.ep1_name.shift()) &
                               (api_events.game_seconds==api_events.game_seconds.shift()),
                                1, 0)))

        api_events = api_events.assign(version = 
                               (np.where(
                               (api_events.event==api_events.event.shift(2)) & 
                               (api_events.ep1_name==api_events.ep1_name.shift(2)) &
                               (api_events.game_seconds==api_events.game_seconds.shift(2) )& 
                               (~api_events.description.str.contains('Penalty Shot')),
                                2, api_events.version)))

        api_events = api_events.assign(version = 
                               (np.where(
                               (api_events.event==api_events.event.shift(3)) & 
                               (api_events.ep1_name==api_events.ep1_name.shift(3)) &
                               (api_events.game_seconds==api_events.game_seconds.shift(3)),
                                3, api_events.version)))#.drop(columns = 'description')
        
        api_events['ep1_name'] = np.where((api_events.description.str.contains('Too many men')) | (api_events.description.str.contains('unsportsmanlike conduct-bench')), 'BENCH', api_events['ep1_name'])
        
        api_events['ep1_name'] = np.where(api_events['ep1_name'].str.contains('ALEXANDRE '), 
                                api_events['ep1_name'].str.replace('ALEXANDRE ', 'ALEX '),
                                api_events['ep1_name'])
    
        api_events['ep1_name'] = np.where(api_events['ep1_name'].str.contains('ALEXANDER '), 
                                    api_events['ep1_name'].str.replace('ALEXANDER ', 'ALEX '),
                                    api_events['ep1_name'])

        api_events['ep1_name'] = np.where(api_events['ep1_name'].str.contains('CHRISTOPHER '), 
                                    api_events['ep1_name'].str.replace('CHRISTOPHER ', 'CHRIS '),
                                    api_events['ep1_name'])
        
        api_events = api_events.assign(
        ep1_name = 
        (np.where(api_events['ep1_name']=="ALEX PECHURSKIY", "ALEX PECHURSKI", 
        (np.where(api_events['ep1_name']=="BEN ONDRUS", "BENJAMIN ONDRUS", 
        (np.where(api_events['ep1_name']=="BRYCE VAN BRABANT", "BRYCE VAN BRABANT", 
        (np.where(api_events['ep1_name']=="CALVIN DE HAAN", "CALVIN DE HAAN", 
        (np.where(api_events['ep1_name']=="CHASE DE LEO", "CHASE DE LEO", 
        (np.where(api_events['ep1_name']=="CAL PETERSEN", "CALVIN PETERSEN",
        (np.where(api_events['ep1_name']=="DANIEL CARCILLO", "DAN CARCILLO", 
        (np.where(api_events['ep1_name']=="DANNY O'REGAN", "DANIEL O'REGAN", 
        (np.where(api_events['ep1_name']=="DAVID VAN DER GULIK", "DAVID VAN DER GULIK", 
        (np.where(api_events['ep1_name']=="EVGENII DADONOV", "EVGENY DADONOV", 
        (np.where(api_events['ep1_name']=="FREDDY MODIN", "FREDRIK MODIN", 
        (np.where(api_events['ep1_name']=="GREG DE VRIES", "GREG DE VRIES", 
        (np.where(api_events['ep1_name']=="ILYA ZUBOV", "ILJA ZUBOV", 
        (np.where(api_events['ep1_name']=="JACOB DE LA ROSE", "JACOB DE LA ROSE", 
        (np.where(api_events['ep1_name']=="JAMES VAN RIEMSDYK", "JAMES VAN RIEMSDYK", 
        (np.where(api_events['ep1_name']=="JEAN-FRANCOIS JACQUES", "J-F JACQUES", 
        (np.where(api_events['ep1_name']=="JAKOB FORSBACKA KARLSSON", "JAKOB FORSBACKA KARLSSON", 
        (np.where(api_events['ep1_name']=="JIM DOWD", "JAMES DOWD", 
        (np.where(api_events['ep1_name']=="JEFF HAMILTON", "JEFFREY HAMILTON", 
        (np.where(api_events['ep1_name']=="JEFF PENNER", "JEFFREY PENNER", 
        (np.where(api_events['ep1_name']=="JOEL ERIKSSON EK", "JOEL ERIKSSON EK", 
        (np.where(api_events['ep1_name']=="MARK VAN GUILDER", "MARK VAN GUILDER", 
        (np.where(api_events['ep1_name']=="MARTIN ST LOUIS", "MARTIN ST. LOUIS", 
        (np.where(api_events['ep1_name']=="MARTIN ST PIERRE", "MARTIN ST. PIERRE", 
        (np.where(api_events['ep1_name']=="MARTIN ST PIERRE", "MARTIN ST. PIERRE", 
        (np.where(api_events['ep1_name']=="MICHAEL CAMMALLERI", "MIKE CAMMALLERI", 
        (np.where(api_events['ep1_name']=="MICHAEL DAL COLLE", "MICHAEL DAL COLLE", 
        (np.where(api_events['ep1_name']=="MICHAEL DEL ZOTTO", "MICHAEL DEL ZOTTO", 
        (np.where(api_events['ep1_name']=="MIKE VERNACE", "MICHAEL VERNACE", 
        (np.where(api_events['ep1_name']=="MIKE YORK", "MICHAEL YORK", 
        (np.where(api_events['ep1_name']=="MIKE VAN RYN", "MIKE VAN RYN", 
        (np.where(api_events['ep1_name']=="MITCHELL MARNER", "MITCH MARNER", 
        (np.where(api_events['ep1_name']=="PAT MAROON", "PATRICK MAROON", 
        (np.where(api_events['ep1_name']=="PA PARENTEAU", "P.A. PARENTEAU", 
        (np.where(api_events['ep1_name']=="PHILLIP DI GIUSEPPE", "PHILLIP DI GIUSEPPE", 
        (np.where(api_events['ep1_name']=="STEFAN DELLA ROVERE", "STEFAN DELLA ROVERE", 
        (np.where(api_events['ep1_name']=="STEPHANE DA COSTA", "STEPHANE DA COSTA", 
        (np.where(api_events['ep1_name']=="TJ GALIARDI", "T.J. GALIARDI", 
        (np.where(api_events['ep1_name']=="TOBY ENSTROM", "TOBIAS ENSTROM",  
        (np.where(api_events['ep1_name']=="TREVOR VAN RIEMSDYK", "TREVOR VAN RIEMSDYK", 
        (np.where(api_events['ep1_name']=="ZACK FITZGERALD", "ZACH FITZGERALD", 

        ## NEW CHANGES
        (np.where(api_events['ep1_name']=="TIM GETTINGER", "TIMOTHY GETTINGER", 
        (np.where(api_events['ep1_name']=="THOMAS DI PAULI", "THOMAS DI PAULI", 
        (np.where(api_events['ep1_name']=="NICHOLAS SHORE", "NICK SHORE",
        (np.where(api_events['ep1_name']=="T.J.  TYNAN", "TJ TYNAN",

        ## '20-21 CHANGES (from HTM update function)
        (np.where(api_events['ep1_name']=="ALEXIS LAFRENI?RE", "ALEXIS LAFRENIÈRE",
        (np.where(api_events['ep1_name']=="ALEXIS LAFRENIERE", "ALEXIS LAFRENIÈRE",
        (np.where(api_events['ep1_name']=="TIM STUTZLE", "TIM STÜTZLE",
        (np.where(api_events['ep1_name']=="TIM ST?TZLE", "TIM STÜTZLE",
        (np.where(api_events['ep1_name']=="EGOR SHARANGOVICH", "YEGOR SHARANGOVICH",
        (np.where(api_events['ep1_name']=="CALLAN FOOTE", "CAL FOOTE",
        (np.where(api_events['ep1_name']=="JOSH DUNNE", "JOSHUA DUNNE", api_events['ep1_name']
        ))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))
        )))))))))))))))))))))))))))))))))))))))))))))

        if drop_description == True:
        
            return api_events.loc[:, ['game_seconds', 'event', 'coords_x', 'coords_y', 'ep1_name', 'period', 'version']].rename(columns = {'ep1_name':'event_player_1'})
        
        else:
            
            return api_events.loc[:, ['game_seconds', 'event', 'coords_x', 'coords_y', 'ep1_name', 'period', 'version', 'description']].rename(columns = {'ep1_name':'event_player_1'})
        
    else:
        print("This game doesn't exist within the API.")
        raise KeyError

# SCRAPE HTML EVENTS

In [52]:
def scrape_html_events(season, game_id):
    #global game
    url = 'http://www.nhl.com/scores/htmlreports/' + season + '/PL0' + game_id + '.HTM'
    page = requests.get(url)
    #if int(season)<20092010:
     #   soup = BeautifulSoup(page.content, 'html.parser')
    #else:
     #   soup = BeautifulSoup(page.content, 'lxml')
    soup = BeautifulSoup(page.content.decode('ISO-8859-1'), 'lxml')
    tds = soup.find_all("td", {"class": re.compile('.*bborder.*')})
    #global stripped_html
    #global eventdf
    stripped_html = hs_strip_html(tds)
    length = int(len(stripped_html)/8)
    eventdf = pd.DataFrame(np.array(stripped_html).reshape(length, 8)).rename(
    columns = {0:'index', 1:'period', 2:'strength', 3:'time', 4:'event', 5:'description', 6:'away_skaters', 7:'home_skaters'})
    split = eventdf.time.str.split(':')
    game_date = soup.find_all('td', {'align':'center', 'style':'font-size: 10px;font-weight:bold'})[2].get_text()
    
    potentialnames = soup.find_all('td', {'align':'center', 'style':'font-size: 10px;font-weight:bold'})
    
    for i in range(0, 999):
        away = potentialnames[i].get_text()
        if ('Away Game') in away or ('tr./Away') in away:
            away = re.split('Match|Game', away)[0]
            break
        
    for i in range(0, 999):
        home = potentialnames[i].get_text()
        if ('Home Game') in home or ('Dom./Home') in home:
            home = re.split('Match|Game', home)[0]
            break
            
    game = eventdf.assign(away_skaters = eventdf.away_skaters.str.replace('\n', ''),
                  home_skaters = eventdf.home_skaters.str.replace('\n', ''),
                  original_time = eventdf.time,
                  time = split.str[0] + ":" + split.str[1].str[:2],
                  home_team = home,
                  away_team = away)
    
    game = game.assign(away_team_abbreviated = game.away_skaters[0].split(' ')[0],
                       home_team_abbreviated = game.home_skaters[0].split(' ')[0])
    
    game = game[game.period!='Per']
    
    game = game.assign(index = game.index.astype(int)).rename(columns = {'index':'event_index'})
    
    game = game.assign(event_team = game.description.str.split(' ').str[0])
    
    game = game.assign(event_team = game.event_team.str.split('\xa0').str[0])
    
    game = game.assign(event_team = np.where(~game.event_team.isin([game.home_team_abbreviated.iloc[0], game.away_team_abbreviated.iloc[0]]), '', game.event_team))
    
    game = game.assign(other_team = np.where(game.event_team=='', '',
                                            np.where(game.event_team==game.home_team_abbreviated.iloc[0], game.away_team_abbreviated.iloc[0], game.home_team_abbreviated.iloc[0])))
    
    game['event_player_str'] = game.description.apply(
    lambda x: re.findall('(#)(\d\d)|(#)(\d)|(-) (\d\d)|(-) (\d)', x)).astype(str
                                                        ).str.replace('#', '').str.replace('-', '').str.replace("'", '').str.replace(',', '').str.replace('(', '').str.replace(')', '').astype(str
                                                        ).str.replace('[', '').str.replace(']', '').apply(lambda x: re.sub(' +', ' ', x)).str.strip()

    game = game.assign(event_player_1 = 
            game.event_player_str.str.split(' ').str[0],
            event_player_2 = 
            game.event_player_str.str.split(' ').str[1],
            event_player_3 = 
            game.event_player_str.str.split(' ').str[2])
    #return game

    if len(game[game.description.str.contains('Drawn by')])>0:
    
        game = game.assign(event_player_2 = np.where(game.description.str.contains('Drawn By'), 
                                          game.description.str.split('Drawn By').str[1].str.split('#').str[1].str.split(' ').str[0].str.strip(), 
                                          game.event_player_2),
                          event_player_3 = np.where(game.description.str.contains('Served By'),
                                                   np.nan,
                                                   game.event_player_3))

    game = game.assign(event_player_1 = np.where((~pd.isna(game.event_player_1)) & (game.event_player_1!=''),
                              np.where(game.event=='FAC', game.away_team_abbreviated,
                                       game.event_team) + (game.event_player_1.astype(str)), 
                              game.event_player_1),
                  event_player_2 = np.where((~pd.isna(game.event_player_2)) & (game.event_player_2!=''),
                              np.where(game.event=='FAC', game.home_team_abbreviated,
                                       np.where(game.event.isin(['BLOCK', 'HIT', 'PENL']), game.other_team, game.event_team)) + (game.event_player_2.astype(str)), 
                              game.event_player_2),
                  event_player_3 = np.where((~pd.isna(game.event_player_3)) & (game.event_player_3!=''),
                              game.event_team + (game.event_player_3.astype(str)), 
                              game.event_player_3))
    
    game = game.assign(
        event_player_1 = np.where((game.event=='FAC') & (game.event_team==game.home_team_abbreviated),
                                 game.event_player_2, game.event_player_1),
        event_player_2 = np.where((game.event=='FAC') & (game.event_team==game.home_team_abbreviated),
                                 game.event_player_1, game.event_player_2))
    
    #return game
    
    roster = scrape_html_roster(season, game_id).rename(columns = {'Nom/Name':'Name'})
    roster = roster[roster.status=='player']
    roster = roster.assign(team_abbreviated = np.where(roster.team=='home', 
                                                       game.home_team_abbreviated.iloc[0],
                                                      game.away_team_abbreviated.iloc[0]))

    roster = roster.assign(teamnum = roster.team_abbreviated + roster['#'],
                          Name = roster.Name.str.split('(').str[0].str.strip())
    
    event_player_1s = roster.loc[:, ['teamnum', 'Name']].rename(columns = {'teamnum':'event_player_1', 'Name':'ep1_name'})
    event_player_2s = roster.loc[:, ['teamnum', 'Name']].rename(columns = {'teamnum':'event_player_2', 'Name':'ep2_name'})
    event_player_3s = roster.loc[:, ['teamnum', 'Name']].rename(columns = {'teamnum':'event_player_3', 'Name':'ep3_name'})
    
    game = game.merge(
    event_player_1s, on = 'event_player_1', how = 'left').merge(
    event_player_2s, on = 'event_player_2', how = 'left').merge(
    event_player_3s, on = 'event_player_3', how = 'left').assign(
    date = game_date)
    #return game
    
    game['period'] = game.period.astype(int)

    game['period_seconds'] = game.time.str.split(':').str[0].str.replace('-', '').astype(int) * 60 + game.time.str.split(':').str[1].str.replace('-', '').astype(int)

    game['game_seconds'] = (np.where(game.period<5, 
                                       (((game.period - 1) * 1200) + game.period_seconds),
                              3900))
    
    game = game.assign(priority = np.where(game.event.isin(['TAKE', 'GIVE', 'MISS', 'HIT', 'SHOT', 'BLOCK']), 1, 
                                            np.where(game.event=="GOAL", 2,
                                                np.where(game.event=="STOP", 3,
                                                    np.where(game.event=="DELPEN", 4,
                                                        np.where(game.event=="PENL", 5,
                                                            np.where(game.event=="CHANGE", 6,
                                                                np.where(game.event=="PEND", 7,
                                                                    np.where(game.event=="GEND", 8,
                                                                        np.where(game.event=="FAC", 9, 0)))))))))).sort_values(by = ['game_seconds', 'period', 'event_player_1', 'event'])
    game = game.assign(version = 
                       (np.where(
                       (game.event==game.event.shift()) & 
                       (game.event_player_1==game.event_player_1.shift()) &
                       (game.event_player_1!='') &
                       (game.game_seconds==game.game_seconds.shift()),
                        1, 0)))
    
    game = game.assign(version = 
                           (np.where(
                           (game.event==game.event.shift(2)) & 
                           (game.event_player_1==game.event_player_1.shift(2)) &
                           (game.game_seconds==game.game_seconds.shift(2)) & 
                           (game.event_player_1!='') &
                           (~game.description.str.contains('Penalty Shot')),
                            2, game.version)))
    
    game = game.assign(version = 
                           (np.where(
                           (game.event==game.event.shift(3)) & 
                           (game.event_player_1==game.event_player_1.shift(3)) &
                           (game.game_seconds==game.game_seconds.shift(3)) & 
                           (game.event_player_1!=''),
                            3, game.version)))
    
    game = game.assign(date = pd.to_datetime(game.date[~pd.isna(game.date)].iloc[0])
                  ).rename(columns = {'date':'game_date'}).sort_values(by = ['event_index'])
    
    game = game.assign(event_player_1 = game.ep1_name, event_player_2 = game.ep2_name, event_player_3 = game.ep3_name).drop(columns = ['ep1_name', 'ep2_name', 'ep3_name'])
    
    game = game.assign(home_team = np.where(game.home_team=='CANADIENS MONTREAL', 'MONTREAL CANADIENS', game.home_team),
                      away_team = np.where(game.away_team=='CANADIENS MONTREAL', 'MONTREAL CANADIENS', game.away_team))
    
    game = game[game.game_seconds<4000]
    
    game['game_date'] = np.where((season=='20072008') & (game_id == '20003'), game.game_date + pd.Timedelta(days=1), game.game_date)
    
    game = game.assign(event_player_1 = np.where((game.description.str.upper().str.contains('TEAM')) | (game.description.str.lower().str.contains('bench')),
                                     'BENCH',
                                     game.event_player_1))
    
    game = game.assign(home_skater_count_temp = (game.home_skaters.apply(lambda x: len(re.findall('[A-Z]', x)))),
          away_skater_count_temp = (game.away_skaters.apply(lambda x: len(re.findall('[A-Z]', x))))
         )
    
    game = game.assign(event_team = np.where((game.event=='PENL') & (game.event_team=='') & (game.description.str.lower().str.contains('bench')) & (game.home_skater_count_temp>game.home_skater_count_temp.shift(-1)),
                                game.home_team_abbreviated, game.event_team))

    game = game.assign(event_team = np.where((game.event=='PENL') & (game.event_team=='') & (game.description.str.lower().str.contains('bench')) & (game.away_skater_count_temp>game.away_skater_count_temp.shift(-1)),
                                game.away_team_abbreviated, game.event_team))
    
    return game.drop(columns = ['period_seconds', 'time', 'priority', 'home_skater_count_temp', 'away_skater_count_temp'])

In [53]:
game = scrape_html_events('20202021', '20215')

# Scrape ESPN XML Events

In [54]:
def scrape_espn_events(espn_game_id, drop_description = True):
    
    ### NEED TO FIX PENALTY SHOTS ##
    
    # Hawks ID: 270114004
    # Sharks ID: 401272106
    # Habs game: 401044320
    # Flames game (first goal unasssisted): 401320053

    url = 'https://www.espn.com/nhl/gamecast/data/masterFeed?lang=en&isAll=true&rand=0&gameId=' + str(espn_game_id)
    page = requests.get(url, timeout = 500)
    try:
        dictionary = xmltodict.parse(page.content.decode('ISO-8859-1'))
    except ExpatError as e:
        problem = int(str(e).split('column')[1].strip())
        problem_value = page.content.decode('ISO-8859-1')[problem]
        dictionary = xmltodict.parse(page.content.decode('ISO-8859-1').replace(problem_value, ''))
    if (dictionary['NHLGamecast']['Plays']) is None:
        raise IndexError('This game has no events.')
    playdict = (dictionary['NHLGamecast']['Plays']['Play'])
    global play_list
    play_list = []
    play_id_list = []

    for i in range(0, len(playdict)):
        play_list.append(playdict[i]['#text'])
        play_id_list.append(playdict[i]['@id'])

    x_coordinates = []
    y_coordinates = []
    game_mins = []
    game_secs = []
    game_pd = []
    event_desc = []

    for i in range (0, len(play_list)):
        split = play_list[i].split('~')
        x_coordinates.append(split[0])
        y_coordinates.append(split[1])
        game_mins.append(play_list[i].split(':')[0].split('~')[-1])
        game_secs.append(play_list[i].split(':')[1].split('~')[0].split('-')[0])
        event_desc.append(" ".join(re.findall("[a-z-'.A-Z]+|\dst|\drd|\d2nd|\d  minutes|\d minutes", play_list[i])))
        if (len(re.split(r'(:\d+)~', play_list[i])))>1:
            game_pd.append((re.split(r'(:\d+)~', play_list[i])[2][0]))
        else:
            game_pd.append(re.split('-\d~|-\d:\d-\d~', play_list[i])[1][0])
            
            
    #event_desc.append(" ".join(re.findall("[a-zA-Z]+", play_list[i])))
    # Below is the code to get information that includes period number and penalty minutes. It is timely and unncessary.
    
    
    espn_events = pd.DataFrame()
    
    #for i in range(0, len(game_secs)):
     #   print((int(game_secs[i])))

    espn_events = espn_events.assign(
    coords_x = x_coordinates,
    coords_y = y_coordinates,
    period = game_pd,
    minutes = game_mins,
    seconds = game_secs,
    description = event_desc)

    espn_events = espn_events.assign(
    coords_x = espn_events.coords_x.astype(int),
    coords_y = espn_events.coords_y.astype(int),
    period = espn_events.period.astype(int),
    minutes = espn_events.minutes.astype(int),
    seconds = espn_events.seconds.astype(int),
    description = espn_events.description.str.strip('-|- -').str.strip()).sort_values(by = ['period', 'minutes', 'seconds'])
    espn_events['minutes'] = np.where(espn_events.minutes<0, 0, espn_events.minutes)
    
    espn_events['duplicated_description'] = espn_events['description']

    espn_events['duplicated_description'] = espn_events['description']

    espn_events['event_player_1'] = espn_events['duplicated_description'].apply(
        lambda x: x.split('Giveaway by')[1].split(' in')[0].strip() if 'Giveaway by' in x else x)
    espn_events['duplicated_description'] = np.where(espn_events['event_player_1']!=espn_events['duplicated_description'], espn_events['event_player_1'], espn_events['duplicated_description'])

    espn_events['event_player_1'] = espn_events['duplicated_description'].apply(
        lambda x: x.split('Takeaway by')[1].split(' in')[0].strip() if 'Takeaway by' in x else x)
    espn_events['duplicated_description'] = np.where(espn_events['event_player_1']!=espn_events['duplicated_description'], espn_events['event_player_1'], espn_events['duplicated_description'])

    espn_events['event_player_1'] = espn_events['duplicated_description'].apply(
        lambda x: x.split('credited with hit')[0].split('credited')[0].strip() if 'credited with hit' in x else x)
    espn_events['duplicated_description'] = np.where(espn_events['event_player_1']!=espn_events['duplicated_description'], espn_events['event_player_1'], espn_events['duplicated_description'])

    espn_events['event_player_1'] = espn_events['duplicated_description'].apply(
        lambda x: x.split('won faceoff')[0].strip() if 'faceoff' in x else x)
    espn_events['duplicated_description'] = np.where(espn_events['event_player_1']!=espn_events['duplicated_description'], espn_events['event_player_1'], espn_events['duplicated_description'])

    espn_events['event_player_1'] = espn_events['duplicated_description'].apply(
        lambda x: re.split('Wristshot|Tip-In|Snapshot|Backhand|Slapshot|Deflection|Wraparound',
                           re.split('scored by|Scored by', x)[1].split('assisted by')[0].split('unassisted')[0].split('Power')[0].split('Empty')[0].split('Shorthanded')[0])[0].strip() if (
            'Goal Scored' in x or 'Goal scored' in x or 'Shootout GOAL' in x) and x!='Goal scored' else x)
    espn_events['duplicated_description'] = np.where(espn_events['event_player_1']!=espn_events['duplicated_description'], espn_events['event_player_1'], espn_events['duplicated_description'])

    espn_events['event_player_1'] = espn_events['duplicated_description'].apply(
        lambda x: re.split('Shot blocked by', x)[1].strip() if 'Shot blocked by' in x else x)
    espn_events['duplicated_description'] = np.where(espn_events['event_player_1']!=espn_events['duplicated_description'], espn_events['event_player_1'], espn_events['duplicated_description'])
    
    espn_events['event_player_1'] = espn_events['duplicated_description'].apply(
        lambda x: re.split('shot blocked', x)[0].strip() if 'blocked' in x else x)
    espn_events['duplicated_description'] = np.where(espn_events['event_player_1']!=espn_events['duplicated_description'], espn_events['event_player_1'], espn_events['duplicated_description'])

    espn_events['event_player_1'] = espn_events['duplicated_description'].apply(
        lambda x: re.split('Shot blocked', x)[1].strip() if 'block' in x else x)
    espn_events['duplicated_description'] = np.where(espn_events['event_player_1']!=espn_events['duplicated_description'], espn_events['event_player_1'], espn_events['duplicated_description'])
    
    espn_events['event_player_1'] = espn_events['duplicated_description'].apply(
        lambda x: re.split('missed by', x)[1].split('Wide')[0].split('Over')[0].split('Goalpost')[0].split('Hit')[0].strip() if 'missed by' in x else x)
    espn_events['duplicated_description'] = np.where(espn_events['event_player_1']!=espn_events['duplicated_description'], espn_events['event_player_1'], espn_events['duplicated_description'])

    espn_events['event_player_1'] = espn_events['duplicated_description'].apply(
        lambda x: re.split('Wristshot|Tip-In|Snapshot|Backhand|Slapshot|Deflection|Saved|Wraparound', re.split('Shot on goal by', x)[1].split('saved')[0].split('ft')[0].split('shootout')[0] if 'Shot on goal' in x and x!='Shot on goal' else x)[0].strip())
    espn_events['duplicated_description'] = np.where(espn_events['event_player_1']!=espn_events['duplicated_description'], espn_events['event_player_1'], espn_events['duplicated_description'])

    espn_events['event_player_1'] = espn_events['duplicated_description'].apply(
        lambda x: re.split('0|2|4|5|10', re.split('Penalty to', x)[1].split('minutes')[0])[0].strip() if 'Penalty to' in x else x)
    espn_events['duplicated_description'] = np.where(espn_events['event_player_1']!=espn_events['duplicated_description'], espn_events['event_player_1'], espn_events['duplicated_description'])

    espn_events['event_player_1'] = espn_events['duplicated_description'].apply(
        lambda x: re.split('saved|MISSES|SAVED', x.split('Shootout attempt by')[1])[0].split('saved')[0].strip() if 'Shootout attempt by' in x else x)
    espn_events['duplicated_description'] = np.where(espn_events['event_player_1']!=espn_events['duplicated_description'], espn_events['event_player_1'], espn_events['duplicated_description'])

    espn_events['event_player_1'] = espn_events['duplicated_description'].apply(
        lambda x: x.split(' on ')[0].strip() if ' on ' in x else x)
    espn_events['duplicated_description'] = np.where(espn_events['event_player_1']!=espn_events['duplicated_description'], espn_events['event_player_1'], espn_events['duplicated_description'])
    
    espn_events['event_player_1'] = espn_events['duplicated_description'].apply(
        lambda x: 'BENCH' if 'Bench' in x and 'Penalty' in x else x)
    espn_events['duplicated_description'] = np.where(espn_events['event_player_1']!=espn_events['duplicated_description'], espn_events['event_player_1'], espn_events['duplicated_description'])
    
    espn_events['event_player_1'] = espn_events['duplicated_description'].apply(
        lambda x: x.split('shootout')[0].strip() if 'shootout attempt against' in x else x)
    espn_events['duplicated_description'] = np.where(espn_events['event_player_1']!=espn_events['duplicated_description'], espn_events['event_player_1'], espn_events['duplicated_description'])
    
    espn_events = espn_events.assign(event_type = np.where((espn_events.description.str.contains("Penalty")) | ((espn_events.description.str.contains("Bench penalty"))),
    "PENL",
    (np.where(((espn_events.description.str.contains("Shot on goal")) | 
    (espn_events.description.str.contains('Shootout attempt by *.* saved')) |
    (espn_events.description.str.contains('Shootout attempt by *.* SAVED')) |
    (espn_events.description.str.contains("shootout attempt *.* results in a SAVE"))),
    "SHOT",
    (np.where(((espn_events.description.str.contains("Shot missed")) |
    (espn_events.description.str.contains('Shootout attempt by *.* MISSES')) |
    (espn_events.description.str.contains("shootout attempt *.* results in a MISS"))),
    "MISS",
    (np.where(espn_events.description.str.contains("faceoff"),
    "FAC",
    (np.where(espn_events.description.str.contains("blocked"),
    "BLOCK",
    (np.where(espn_events.description.str.contains("credited with hit"),
    "HIT",
    (np.where((espn_events.description.str.contains("Giveaway by")) | (espn_events.description.str.contains("Giveaway in")),
    "GIVE",
    (np.where((espn_events.description.str.contains("Takeaway by")) | (espn_events.description.str.contains("Takeaway in")),
    "TAKE",
    (np.where(espn_events.description.str.contains("Goal Scored|Goal scored|GOAL scored|shootout attempt *.* results in a GOAL"),
    "GOAL",
    (np.where(espn_events.description.str.contains("Start of"),
    "PSTR",
    (np.where((espn_events.description.str.contains("End of")) & 
          (espn_events.description.str!="End of Game"),
    "PEND",
    (np.where(espn_events.description.str.contains("Stoppage"),
     "STOP",  
        (np.where(espn_events.description=='End of Game',
         "GEND",  
    '\xa0'))))))))))))))))))))))))))#.loc[:, ['game_seconds', 'description','event_type',  'coords_x', 'coords_y', 'event_player_1']]
    espn_events = espn_events.assign(
        event_player_1 = np.where(espn_events.event_player_1==espn_events.description, '\xa0', espn_events.event_player_1))
    
    espn_events = espn_events.assign(priority = np.where(espn_events.event_type.isin(['TAKE', 'GIVE', 'MISS', 'HIT', 'SHOT', 'BLOCK']), 1, 
                                            np.where(espn_events.event_type=="GOAL", 2,
                                                np.where(espn_events.event_type=="STOP", 3,
                                                    np.where(espn_events.event_type=="DELPEN", 4,
                                                        np.where(espn_events.event_type=="PENL", 5,
                                                            np.where(espn_events.event_type=="CHANGE", 6,
                                                                np.where(espn_events.event_type=="PEND", 7,
                                                                    np.where(espn_events.event_type=="GEND", 8,
                                                                        np.where(espn_events.event_type=="FAC", 9, 0))))))))),
                                    event_player_1 = espn_events.event_player_1.str.upper(),
                                    game_seconds = np.where(espn_events.period<5, 
                                    ((espn_events.period - 1) * 1200) + (espn_events.minutes * 60) + espn_events.seconds, 3900))
    espn_events['game_seconds'] = espn_events.game_seconds.astype(int)
    espn_events = espn_events.sort_values(by = ['period', 'game_seconds', 'event_player_1', 'priority']).rename(
    columns = {'event_type':'event'}).loc[:, ['coords_x', 'coords_y', 'event_player_1', 'event', 'game_seconds', 'description', 'period']]
    
    espn_events['event_player_1'] = np.where(espn_events['event_player_1'].str.contains('ALEXANDRE '), 
                                espn_events['event_player_1'].str.replace('ALEXANDRE ', 'ALEX '),
                                espn_events['event_player_1'])
    
    espn_events['event_player_1'] = np.where(espn_events['event_player_1'].str.contains('ALEXANDER '), 
                                espn_events['event_player_1'].str.replace('ALEXANDER ', 'ALEX '),
                                espn_events['event_player_1'])
    
    espn_events['event_player_1'] = np.where(espn_events['event_player_1'].str.contains('CHRISTOPHER '), 
                                espn_events['event_player_1'].str.replace('CHRISTOPHER ', 'CHRIS '),
                                espn_events['event_player_1'])
    
    espn_events = espn_events.assign(event_player_1 = 
    np.where(espn_events.event_player_1=='PATRICK MAROON', 'PAT MAROON',
    (np.where(espn_events.event_player_1=='J T COMPHER', 'J.T. COMPHER', 
    (np.where(espn_events.event_player_1=='J T MILLER', 'J.T. MILLER', 
    (np.where(espn_events.event_player_1=='T J OSHIE', 'T.J. OSHIE', 
    (np.where((espn_events.event_player_1=='ALEXIS LAFRENIERE') | (espn_events.event_player_1=='ALEXIS LAFRENI RE'), 'ALEXIS LAFRENIÈRE', 
    (np.where((espn_events.event_player_1=='TIM STUTZLE') | (espn_events.event_player_1=='TIM ST TZLE'), 'TIM STÜTZLE',
    (np.where(espn_events.event_player_1=='T.J. BRODIE', 'TJ BRODIE',
    (np.where(espn_events.event_player_1=='MATTHEW IRWIN', 'MATT IRWIN',
    (np.where(espn_events.event_player_1=='STEVE KAMPFER', 'STEVEN KAMPFER',
    (np.where(espn_events.event_player_1=='STEVE KAMPFER', 'STEVEN KAMPFER',
    (np.where(espn_events.event_player_1=='JEFFREY TRUCHON-VIEL', 'JEFFREY VIEL',
    (np.where(espn_events.event_player_1=='ZACHARY JONES', 'ZAC JONES',
    (np.where(espn_events.event_player_1=='MITCH MARNER', 'MITCHELL MARNER',
    (np.where(espn_events.event_player_1=='MATHEW DUMBA', 'MATT DUMBA',
    (np.where(espn_events.event_player_1=='JOSHUA MORRISSEY', 'JOSH MORRISSEY',
    (np.where(espn_events.event_player_1=='P K SUBBAN', 'P.K. SUBBAN',
    (np.where(espn_events.event_player_1=='EGOR SHARANGOVICH', 'YEGOR SHARANGOVICH',
    (np.where(espn_events.event_player_1=='MAXIME COMTOIS', 'MAX COMTOIS',
    (np.where(espn_events.event_player_1=='NICHOLAS CAAMANO', 'NICK CAAMANO',
    (np.where(espn_events.event_player_1=='DANIEL CARCILLO', 'DAN CARCILLO',
    (np.where(espn_events.event_player_1=='ALEXANDER OVECHKIN', 'ALEX OVECHKIN',
    (np.where(espn_events.event_player_1=='MICHAEL CAMMALLERI', 'MIKE CAMMALLERI',
    (np.where(espn_events.event_player_1=='DAVE STECKEL', 'DAVID STECKEL',
    (np.where(espn_events.event_player_1=='JIM DOWD', 'JAMES DOWD', 
    (np.where(espn_events.event_player_1=='MAXIME TALBOT', 'MAX TALBOT',
    (np.where(espn_events.event_player_1=='MIKE ZIGOMANIS', 'MICHAEL ZIGOMANIS',
    (np.where(espn_events.event_player_1=='VINNY PROSPAL', 'VACLAV PROSPAL',
    (np.where(espn_events.event_player_1=='MIKE YORK', 'MICHAEL YORK',
    (np.where(espn_events.event_player_1=='JACOB DOWELL', 'JAKE DOWELL',
    (np.where(espn_events.event_player_1=='MICHAEL RUPP', 'MIKE RUPP',
    (np.where(espn_events.event_player_1=='ALEXEI KOVALEV', 'ALEX KOVALEV',
    (np.where(espn_events.event_player_1=='SLAVA KOZLOV', 'VYACHESLAV KOZLOV',
    (np.where(espn_events.event_player_1=='JEFF HAMILTON', 'JEFFREY HAMILTON',
    (np.where(espn_events.event_player_1=='JOHNNY POHL', 'JOHN POHL',
    (np.where(espn_events.event_player_1=='DANIEL GIRARDI', 'DAN GIRARDI',
    (np.where(espn_events.event_player_1=='NIKOLAI ZHERDEV', 'NIKOLAY ZHERDEV',
    (np.where(espn_events.event_player_1=='J.P. DUMONT', 'J-P DUMONT',
    (np.where(espn_events.event_player_1=='DWAYNE KING', 'DJ KING',
    (np.where(espn_events.event_player_1=='JOHN ODUYA', 'JOHNNY ODUYA',
    (np.where(espn_events.event_player_1=='ROBERT SCUDERI', 'ROB SCUDERI',
    (np.where(espn_events.event_player_1=='DOUG MURRAY', 'DOUGLAS MURRAY',
    (np.where(espn_events.event_player_1=='VACLAV PROSPAL', 'VINNY PROSPAL',
    (np.where(espn_events.event_player_1=='RICH PEVERLY', 'RICH PEVERLEY',
    espn_events.event_player_1.str.strip()
             ))))))))))))))))))))))))))))))))))))))))))))
             ))))))))))))))))))))))))))))))))))))))))))
    
    espn_events = espn_events.assign(version = 
                       (np.where(
                       (espn_events.event==espn_events.event.shift()) & 
                       (espn_events.event_player_1==espn_events.event_player_1.shift()) &
                       (espn_events.event_player_1!='') &
                       (espn_events.game_seconds==espn_events.game_seconds.shift()),
                        1, 0)))
    
    espn_events = espn_events.assign(version = 
                           (np.where(
                           (espn_events.event==espn_events.event.shift(2)) & 
                           (espn_events.event_player_1==espn_events.event_player_1.shift(2)) &
                           (espn_events.game_seconds==espn_events.game_seconds.shift(2)) & 
                           (espn_events.event_player_1!='') &
                           (~espn_events.description.str.contains('Penalty Shot')),
                            2, espn_events.version)))
    
    espn_events = espn_events.assign(version = 
                           (np.where(
                           (espn_events.event==espn_events.event.shift(3)) & 
                           (espn_events.event_player_1==espn_events.event_player_1.shift(3)) &
                           (espn_events.game_seconds==espn_events.game_seconds.shift(3)) & 
                           (espn_events.event_player_1!=''),
                            3, espn_events.version)))
    
    espn_events['espn_id'] = int(espn_game_id)
    
    espn_events['event_player_1'] = espn_events['event_player_1'].str.strip()
    
    #espn_events = espn_events.assign(event_player_1 = np.where(
    #espn_events.event_player_1=='ALEX BURROWS', 'ALEXANDRE BURROWS', espn_events.event_player_1))
    
    global look
    look = espn_events
    
    espn_events['coords_x'] = espn_events['coords_x'] + 1
    espn_events['coords_y'] = espn_events['coords_y'] - 1
    
    if drop_description == True:
        return espn_events.drop(columns = 'description')
    else:
        return espn_events

# GET ESPN IDS FROM GAMES

In [56]:
def scrape_espn_ids_single_game(game_date, home_team, away_team):
    gamedays = pd.DataFrame()
    
    if home_team == 'ATLANTA THRASHERS':
        home_team = 'WINNIPEG JETS'
    if away_team == 'ATLANTA THRASHERS':
        away_team = 'WINNIPEG JETS'
        
    if home_team == 'PHOENIX COYOTES':
        home_team = 'ARIZONA COYOTES'
    if away_team == 'PHOENIX COYOTES':
        away_team = 'ARIZONA COYOTES'
    
    this_date = (game_date)
    url = 'http://www.espn.com/nhl/scoreboard?date=' + this_date.replace("-", "")
    page = requests.get(url, timeout = 500)
    soup = BeautifulSoup(page.content, parser = 'lxml')
    soup_found = soup.find_all('a', {'class':['AnchorLink truncate', 'AnchorLink Button Button--sm Button--anchorLink Button--alt mb4 w-100'], 'href':[re.compile("/nhl/team/_/name/"), re.compile("game/_")]})
    at = []
    ht = []
    gids = []
    fax = pd.DataFrame()
    #print(str(i))
    for i in range (0, (int(len(soup_found)/3))):
        away = soup_found[0 + (i * 3)]['href'].rsplit('/')[-1].replace('-', ' ').upper()
        home = soup_found[1 + (i * 3)]['href'].rsplit('/')[-1].replace('-', ' ').upper()
        espnid = soup_found[2 + (i * 3)]['href'].split('gameId/', 1)[1]
        at.append(away)
        ht.append(home)
        gids.append(espnid)

    fax = fax.assign(
    away_team = at,
    home_team = ht,
    espn_id = gids,
    game_date = pd.to_datetime(this_date))

    gamedays = gamedays.append(fax)

    gamedays = gamedays.assign(
        home_team = np.where(gamedays.home_team=='ST LOUIS BLUES', 'ST. LOUIS BLUES', gamedays.home_team),
        away_team = np.where(gamedays.away_team=='ST LOUIS BLUES', 'ST. LOUIS BLUES', gamedays.away_team),
        espn_id = gamedays.espn_id.astype(int))
    
    #gamedays = gamedays.assign(
     #   home_team = np.where(gamedays.home_team=='WINNIPEG JETS', 'ATLANTA THRASHERS', gamedays.home_team),
      #  away_team = np.where(gamedays.away_team=='WINNIPEG JETS', 'ATLANTA THRASHERS', gamedays.away_team),
       # espn_id = gamedays.espn_id.astype(int))
    
    gamedays = gamedays[(gamedays.game_date==this_date) & (gamedays.home_team==home_team) & (gamedays.away_team==away_team)]
        
    return gamedays

# MERGE AND PREPARE EVENTS AND SHIFTS.

In [57]:
def merge_and_prepare(events, shifts):
    
    season = str(int(str(events.game_id.iloc[0])[:4])) + str(int(str(events.game_id.iloc[0])[:4]) + 1)
    small_id = str(events.game_id.iloc[0])[5:]
    game_id = int(events.game_id.iloc[0])
    
    merged = pd.concat([events, shifts])
    
    merged = merged.assign(home_team = merged[~(pd.isna(merged.home_team))].home_team.iloc[0],
                          away_team = merged[~(pd.isna(merged.away_team))].away_team.iloc[0],
                          home_team_abbreviated = merged[~(pd.isna(merged.home_team_abbreviated))].home_team_abbreviated.iloc[0],
                          away_team_abbreviated = merged[~(pd.isna(merged.away_team_abbreviated))].away_team_abbreviated.iloc[0])

    merged = merged.assign(event_team = np.where(merged.team==merged.home_team, merged.home_team_abbreviated, 
                                        np.where(merged.team==merged.away_team, merged.away_team_abbreviated, 
                                                 merged.event_team)))

    merged = merged.assign(event = np.where((pd.isna(merged.event)) & 
                                     ((~pd.isna(merged.number_off)) | (~pd.isna(merged.number_on))), "CHANGE", merged.event))

    home_space = ' ' + merged['home_team_abbreviated'].iloc[0]
    away_space = ' ' + merged['away_team_abbreviated'].iloc[0]

    merged['away_skaters'] = np.where(pd.isna(merged.away_skaters), '\xa0', merged.away_skaters)

    merged['tmp'] = merged.away_skaters.str.replace("[^0-9]", " ")

    merged['tmp2'] = (merged.tmp.str.strip().str.split("  ")).apply(lambda x: natsorted(x)).apply(lambda x: ' '.join(x))

    merged['tmp2'] = (merged.away_team_abbreviated.iloc[0] + merged.tmp2).str.replace(" ", away_space).str.replace(" ", ", ")

    merged['tmp2'] = np.where(merged.tmp2.str.strip()==merged.away_team_abbreviated.iloc[0], '\xa0', merged.tmp2)

    merged['away_on_ice'] = merged['tmp2']

    merged['home_skaters'] = np.where(pd.isna(merged.home_skaters), '\xa0', merged.home_skaters)

    merged['tmp'] = merged.home_skaters.str.replace("[^0-9]", " ")

    merged['tmp2'] = (merged.tmp.str.strip().str.split("  ")).apply(lambda x: natsorted(x)).apply(lambda x: ' '.join(x))

    merged['tmp2'] = (merged.home_team_abbreviated.iloc[0] + merged.tmp2).str.replace(" ", home_space).str.replace(" ", ", ")

    merged['tmp2'] = np.where(merged.tmp2.str.strip()==merged.home_team_abbreviated.iloc[0], '\xa0', merged.tmp2)

    merged['home_on_ice'] = merged['tmp2']

    merged = merged.sort_values(by = ['game_seconds', 'period'])

    merged = merged.assign(jumping_on = (np.where(merged.home_team == merged.team, (merged.home_team_abbreviated.iloc[0] + merged.on_numbers).str.replace(", ", home_space).str.replace(" ", ", "), 
                                   np.where(merged.away_team == merged.team, (merged.away_team_abbreviated.iloc[0] + merged.on_numbers).str.replace(", ", away_space).str.replace(" ", ", "),
                                            np.nan))),
                          jumping_off = (np.where(merged.home_team == merged.team, (merged.home_team_abbreviated.iloc[0] + merged.off_numbers).str.replace(", ", home_space).str.replace(" ", ", "), 
                                   np.where(merged.away_team == merged.team, (merged.away_team_abbreviated.iloc[0] + merged.off_numbers).str.replace(", ", away_space).str.replace(" ", ", "),
                                            np.nan))),
                          prio = np.where(merged.event=="CHANGE", 0,
                                          np.where(merged.event.isin(['PGSTR', 'PGEND', 'PSTR', 'PEND', 'ANTHEM']), -1, 1))).sort_values(
        by = ['game_seconds', 'period', 'event_index'])

    merged = merged.assign(change_before_event = np.where(
        (
            (merged.away_on_ice!='\xa0') & (merged.event.shift()=='CHANGE') & (merged.away_on_ice!=merged.away_on_ice.shift()) | 
            (merged.home_on_ice!='\xa0') & (merged.event.shift()=='CHANGE') & (merged.home_on_ice!=merged.home_on_ice.shift())
        ), 1, 0
    ))

    merged = merged.assign(change_prio = 
                          np.where((merged.team==merged.home_team) & (merged.event=='CHANGE') , 1,
                                  np.where((merged.team==merged.away_team) & (merged.event=='CHANGE'), -1, 0)))

    merged = merged.assign(priority = np.where(merged.event.isin(['TAKE', 'GIVE', 'MISS', 'HIT', 'SHOT', 'BLOCK']), 1, 
                                                np.where(merged.event=="GOAL", 2,
                                                    np.where(merged.event=="STOP", 3,
                                                        np.where(merged.event=="DELPEN", 4,
                                                            np.where(merged.event=="PENL", 5,
                                                                np.where(merged.event=="CHANGE", 6,
                                                                    np.where(merged.event=="PEND", 7,
                                                                        np.where(merged.event=="GEND", 8,
                                                                            np.where(merged.event=="FAC", 9, 0)))))))))).sort_values(by = ['game_seconds', 'period', 'priority', 'event_index', 'change_prio'])

    merged = merged.reset_index(drop = True).reset_index().rename(columns = {'index':'event_index', 'event_index':'original_index'})

    global roster

    roster = scrape_html_roster(season, small_id).rename(columns = {'Nom/Name':'Name'})

    roster = roster.assign(team_abbreviated = np.where(roster.team=='home', 
                                                       merged.home_team_abbreviated.iloc[0],
                                                      merged.away_team_abbreviated.iloc[0]))

    roster = roster.assign(teamnum = roster.team_abbreviated + roster['#'],
                          Name = roster.Name.str.split('(').str[0].str.strip())

    roster = roster.assign(Name = np.where((roster.Name=='SEBASTIAN AHO') &( roster.team_name == 'NEW YORK ISLANDERS'), 'SEBASTIAN AHO (SWE)', roster.Name))

    goalies = roster[(roster.Pos=='G') & (roster.status!='scratch')]

    away_roster = roster[(roster.team=='away') & (roster.status!='scratch')]
    home_roster = roster[(roster.team=='home') & (roster.status!='scratch')]

    merged.jumping_on = np.where(pd.isna(merged.jumping_on), '\xa0', merged.jumping_on)
    merged.jumping_off = np.where(pd.isna(merged.jumping_off), '\xa0', merged.jumping_off)

    awaydf = pd.DataFrame()

    for i in range(0, len(away_roster)):
        vec = pd.DataFrame(
                np.cumsum(
                    (np.where(merged.jumping_on.str.split(', ').apply(lambda x: away_roster.teamnum.iloc[i] in x)==True, 1, 0)) - (
                    np.where((merged.jumping_off.str.split(', ').apply(lambda x: away_roster.teamnum.iloc[i] in x)==True) & (merged.event=='CHANGE'), 1, 0))
                                     ))
        awaydf = pd.concat([awaydf, vec], axis = 1)

    awaydf.columns = away_roster.Name

    global homedf

    homedf = pd.DataFrame()

    for i in range(0, len(home_roster)):
        vec = pd.DataFrame(
                np.cumsum(
                    (np.where(merged.jumping_on.str.split(', ').apply(lambda x: home_roster.teamnum.iloc[i] in x)==True, 1, 0)) - (
                    np.where((merged.jumping_off.str.split(', ').apply(lambda x: home_roster.teamnum.iloc[i] in x)==True) & (merged.event=='CHANGE'), 1, 0))
                                     ))
        homedf = pd.concat([homedf, vec], axis = 1)

    homedf.columns = home_roster.Name

    global home_on
    global away_on

    home_on = pd.DataFrame((homedf==1).apply(lambda y: homedf.columns[y.tolist()].tolist(), axis=1))
    home_on[0] = (home_on[0].apply(','.join)).apply(lambda x: ','.join(natsorted(x.split(','))))

    away_on = pd.DataFrame((awaydf==1).apply(lambda y: awaydf.columns[y.tolist()].tolist(), axis=1))
    away_on[0] = (away_on[0].apply(','.join)).apply(lambda x: ','.join(natsorted(x.split(','))))

    away_on = away_on[0].str.split(',', expand=True).rename(columns = {0:'away_on_1', 1:'away_on_2', 2:'away_on_3', 3:'away_on_4', 4:'away_on_5', 5:'away_on_6', 6:'away_on_7', 7:'away_on_8', 8:'away_on_9'})
    home_on = home_on[0].str.split(',', expand=True).rename(columns = {0:'home_on_1', 1:'home_on_2', 2:'home_on_3', 3:'home_on_4', 4:'home_on_5', 5:'home_on_6', 6:'home_on_7', 7:'home_on_8', 8:'home_on_9'})

    if 'away_on_6' not in away_on:
        away_on['away_on_6'] = np.nan
    if 'away_on_7' not in away_on:
        away_on['away_on_7'] = np.nan
    if 'away_on_8' not in away_on:
        away_on['away_on_8'] = np.nan
    if 'away_on_9' not in away_on:
        away_on['away_on_9'] = np.nan
    if 'home_on_6' not in home_on:
        home_on['home_on_6'] = np.nan
    if 'home_on_7' not in home_on:
        home_on['home_on_7'] = np.nan
    if 'home_on_8' not in home_on:
        home_on['home_on_8'] = np.nan
    if 'home_on_9' not in home_on:
        home_on['home_on_9'] = np.nan

    game = pd.concat([merged, home_on, away_on], axis = 1)

    game = game.assign(
    event_team = np.where(game.event_team==game.home_team, game.home_team_abbreviated,
                         np.where(game.event_team==game.away_team, game.away_team_abbreviated,
                                 game.event_team)),
    description = game.description.astype(str))

    game['description'] = np.where(game.description=='nan', '\xa0', game.description)

    game = game.drop(columns = ['original_index', 'strength', 'original_time', 'home_team', 'away_team', 'other_team', 'event_player_str',
                                'version', 'team', 'change_before_event', 'prio', 'change_prio', 'priority', 'tmp', 'tmp2']).rename(
        columns = {'away_team_abbreviated':'away_team', 'home_team_abbreviated':'home_team', 'coordsx':'coords_x', 'coordsy':'coords_y',
                  'ep1_name':'event_player_1', 'ep2_name':'event_player_2', 'ep3_name':'event_player_3'}).assign(
    game_id = int(game_id),
    season = int(season),
    event_zone = game.description.apply(lambda x: re.search('(\S+?) Zone', x)).apply(lambda x: group_if_not_none(x)),
    event_detail = 
           np.where(game.event.isin(['SHOT', 'BLOCK', 'MISS', 'GOAL']), 
                    game.description.str.split(', ').str[1].str.strip(),
                        np.where(game.event.isin(["PSTR", "PEND", "SOC", "GEND"]),
                        game.description.str.split(': ').str[1].str.strip(),
                                np.where(game.event=='PENL', 
                                game.description.str.split('(').str[1].str.split(')').str[0].str.strip(),
                                        np.where(game.event=='CHANGE',
                                        game.description.str.split(' - ').str[0].str.strip(),
                                            np.where(pd.isna(game.description), '\xa0',
                                                    '\xa0'))))))

    game = game.assign(home_goalie = np.where(
    game.home_on_1.isin(goalies.Name), game.home_on_1,
    np.where(
    game.home_on_2.isin(goalies.Name), game.home_on_2,
    np.where(
    game.home_on_3.isin(goalies.Name), game.home_on_3,
    np.where(
    game.home_on_4.isin(goalies.Name), game.home_on_4,
    np.where(
    game.home_on_5.isin(goalies.Name), game.home_on_5,
    np.where(
    game.home_on_6.isin(goalies.Name), game.home_on_6,
    np.where(
    game.home_on_7.isin(goalies.Name), game.home_on_7,
    np.where(
    game.home_on_8.isin(goalies.Name), game.home_on_8,
    np.where(
    game.home_on_9.isin(goalies.Name), game.home_on_9,
    np.nan))))))))),
    away_goalie = np.where(
    game.away_on_1.isin(goalies.Name), game.away_on_1,
    np.where(
    game.away_on_2.isin(goalies.Name), game.away_on_2,
    np.where(
    game.away_on_3.isin(goalies.Name), game.away_on_3,
    np.where(
    game.away_on_4.isin(goalies.Name), game.away_on_4,
    np.where(
    game.away_on_5.isin(goalies.Name), game.away_on_5,
    np.where(
    game.away_on_6.isin(goalies.Name), game.away_on_6,
    np.where(
    game.away_on_7.isin(goalies.Name), game.away_on_7,
    np.where(
    game.away_on_8.isin(goalies.Name), game.away_on_8,
    np.where(
    game.away_on_9.isin(goalies.Name), game.away_on_9,
    np.nan))))))))))

    game = game.assign(
    away_on_1 = np.where((pd.isna(game.away_on_1)) | (game.away_on_1 is None) | (game.away_on_1=='') | (game.away_on_1=='\xa0'), '\xa0', game.away_on_1),
    away_on_2 = np.where((pd.isna(game.away_on_2)) | (game.away_on_2 is None) | (game.away_on_2=='') | (game.away_on_2=='\xa0'), '\xa0', game.away_on_2),
    away_on_3 = np.where((pd.isna(game.away_on_3)) | (game.away_on_3 is None) | (game.away_on_3=='') | (game.away_on_3=='\xa0'), '\xa0', game.away_on_3),
    away_on_4 = np.where((pd.isna(game.away_on_4)) | (game.away_on_4 is None) | (game.away_on_4=='') | (game.away_on_4=='\xa0'), '\xa0', game.away_on_4),
    away_on_5 = np.where((pd.isna(game.away_on_5)) | (game.away_on_5 is None) | (game.away_on_5=='') | (game.away_on_5=='\xa0'), '\xa0', game.away_on_5),
    away_on_6 = np.where((pd.isna(game.away_on_6)) | (game.away_on_6 is None) | (game.away_on_6=='') | (game.away_on_6=='\xa0'), '\xa0', game.away_on_6),
    away_on_7 = np.where((pd.isna(game.away_on_7)) | (game.away_on_7 is None) | (game.away_on_7=='') | (game.away_on_7=='\xa0'), '\xa0', game.away_on_7),
    away_on_8 = np.where((pd.isna(game.away_on_8)) | (game.away_on_8 is None) | (game.away_on_8=='') | (game.away_on_8=='\xa0'), '\xa0', game.away_on_8),
    away_on_9 = np.where((pd.isna(game.away_on_9)) | (game.away_on_9 is None) | (game.away_on_9=='') | (game.away_on_9=='\xa0'), '\xa0', game.away_on_9),
    home_on_1 = np.where((pd.isna(game.home_on_1)) | (game.home_on_1 is None) | (game.home_on_1=='') | (game.home_on_1=='\xa0'), '\xa0', game.home_on_1),
    home_on_2 = np.where((pd.isna(game.home_on_2)) | (game.home_on_2 is None) | (game.home_on_2=='') | (game.home_on_2=='\xa0'), '\xa0', game.home_on_2),
    home_on_3 = np.where((pd.isna(game.home_on_3)) | (game.home_on_3 is None) | (game.home_on_3=='') | (game.home_on_3=='\xa0'), '\xa0', game.home_on_3),
    home_on_4 = np.where((pd.isna(game.home_on_4)) | (game.home_on_4 is None) | (game.home_on_4=='') | (game.home_on_4=='\xa0'), '\xa0', game.home_on_4),
    home_on_5 = np.where((pd.isna(game.home_on_5)) | (game.home_on_5 is None) | (game.home_on_5=='') | (game.home_on_5=='\xa0'), '\xa0', game.home_on_5),
    home_on_6 = np.where((pd.isna(game.home_on_6)) | (game.home_on_6 is None) | (game.home_on_6=='') | (game.home_on_6=='\xa0'), '\xa0', game.home_on_6),
    home_on_7 = np.where((pd.isna(game.home_on_7)) | (game.home_on_7 is None) | (game.home_on_7=='') | (game.home_on_7=='\xa0'), '\xa0', game.home_on_7),
    home_on_8 = np.where((pd.isna(game.home_on_8)) | (game.home_on_8 is None) | (game.home_on_8=='') | (game.home_on_8=='\xa0'), '\xa0', game.home_on_8),
    home_on_9 = np.where((pd.isna(game.home_on_9)) | (game.home_on_9 is None) | (game.home_on_9=='') | (game.home_on_9=='\xa0'), '\xa0', game.home_on_9),
    home_goalie = np.where(pd.isna(game.home_goalie), '\xa0', game.home_goalie), away_goalie = np.where(pd.isna(game.away_goalie), '\xa0', game.away_goalie)
    )

    game = game.assign(home_skaters = 
                       np.where(game.home_on_1!='\xa0', 1, 0) + np.where(game.home_on_2!='\xa0', 1, 0) + np.where(game.home_on_3!='\xa0', 1, 0) + np.where(game.home_on_4!='\xa0', 1, 0) + 
                       np.where(game.home_on_5!='\xa0', 1, 0) + np.where(game.home_on_6!='\xa0', 1, 0) + np.where(game.home_on_7!='\xa0', 1, 0) + np.where(game.home_on_8!='\xa0', 1, 0) +
                       np.where(game.home_on_9!='\xa0', 1, 0) - np.where((game.home_goalie!='\xa0') & (game.period<5), 1, 0),
                       away_skaters = 
                       np.where(game.away_on_1!='\xa0', 1, 0) + np.where(game.away_on_2!='\xa0', 1, 0) + np.where(game.away_on_3!='\xa0', 1, 0) + np.where(game.away_on_4!='\xa0', 1, 0) + 
                       np.where(game.away_on_5!='\xa0', 1, 0) + np.where(game.away_on_6!='\xa0', 1, 0) + np.where(game.away_on_7!='\xa0', 1, 0) + np.where(game.away_on_8!='\xa0', 1, 0) +
                       np.where(game.away_on_9!='\xa0', 1, 0) - np.where((game.away_goalie!='\xa0') & (game.period<5), 1, 0))

    game = game.assign(home_skater_temp = 
                np.where((game.home_goalie=='\xa0') , 'E', game.home_skaters),
           away_skater_temp = 
                np.where((game.away_goalie=='\xa0') , 'E', game.away_skaters))

    game = game.assign(game_strength_state = (game.home_skater_temp.astype(str)) + 'v' + (game.away_skater_temp.astype(str)),
                      event_zone = np.where(game.event_zone is not None, game.event_zone.str.replace(". Zone", ""), '\xa0'),
                      home_score = np.cumsum(np.where((game.event.shift()=='GOAL') & (game.period<5) & (game.event_team.shift()==game.home_team), 1, 0)),
                      away_score = np.cumsum(np.where((game.event.shift()=='GOAL') & (game.period<5) & (game.event_team.shift()==game.away_team), 1, 0))).drop(
        columns = ['home_skater_temp', 'away_skater_temp'])

    game = game.assign(game_score_state = (game.home_score.astype(str)) + 'v' + (game.away_score.astype(str)),
                      game_date = pd.to_datetime(game.game_date[~pd.isna(game.game_date)].iloc[0])
                      )

    game.number_off = np.where((game.jumping_on!='\xa0') & (game.jumping_off=='\xa0'), 0, game.number_off)
    game.number_on = np.where((game.jumping_off!='\xa0') & (game.jumping_on=='\xa0'), 0, game.number_on)

    so = game[game.period==5]

    if len(so)>0:
        game = game[game.period<5]
        home = roster[roster.team=='home'].rename(columns = {'teamnum':'home_on_ice', 'Name':'home_goalie_name'}).loc[:, ['home_goalie_name', 'home_on_ice']]
        away = roster[roster.team=='away'].rename(columns = {'teamnum':'away_on_ice', 'Name':'away_goalie_name'}).loc[:, ['away_goalie_name', 'away_on_ice']]
        so = so.merge(away, how = 'left', indicator = True).drop(columns = ['_merge']).merge(home, how = 'left')
        so = so.assign(
        home_goalie = so.home_goalie_name,
        away_goalie = so.away_goalie_name).drop(columns = ['away_goalie_name', 'home_goalie_name'])
        so_winner = so[so.event=='GOAL'].groupby('event_team')['event', 'home_team'].count().reset_index().sort_values(by = ['event', 'event_team'],ascending = False).event_team.iloc[0]
        so = so.assign(
            home_on_1 = so.home_goalie,
            away_on_1 = so.away_goalie,
            home_on_2 = np.where(so.event_team==so.home_team, so.event_player_1, '\xa0'),
            away_on_2 = np.where(so.event_team==so.away_team, so.event_player_1, '\xa0'))
        if len(so[so.event=='PEND'])>0:
            end_event = so[so.event=='PEND'].index.astype(int)[0]
            so = so.assign(
            home_score = np.where((so.index>=end_event) & (so_winner == so.home_team), 1+so.home_score, so.home_score),
            away_score = np.where((so.index>=end_event) & (so_winner == so.away_team), 1+so.away_score, so.away_score))
        game = pd.concat([game, so])

    game['event_length'] = game.game_seconds.shift(-1) - game.game_seconds
    game['event_length'] = (np.where((pd.isna(game.event_length)) | (game.event_length<0), 0, game.event_length)).astype(int)
    game['event_index'] = game.event_index + 1
    
    if 'coords_x' and 'coords_y' in game.columns:
    
        columns = ['season', 'game_id', 'game_date', 'event_index',
        'period', 'game_seconds', 'event', 'description',
        'event_detail', 'event_zone', 'event_team', 'event_player_1',
        'event_player_2', 'event_player_3', 'event_length', 'coords_x',
        'coords_y', 'number_on', 'number_off', 'jumping_on', 'jumping_off',
        'home_on_1', 'home_on_2', 'home_on_3', 'home_on_4', 'home_on_5',
        'home_on_6', 'home_on_7', 'home_on_8', 'home_on_9', 'away_on_1', 'away_on_2', 'away_on_3',
        'away_on_4', 'away_on_5', 'away_on_6', 'away_on_7', 'away_on_8', 'away_on_9', 'home_goalie',
        'away_goalie', 'home_team', 'away_team', 'home_skaters', 'away_skaters',
        'home_score', 'away_score', 'game_score_state', 'game_strength_state']
        
    else:
        
        columns = ['season', 'game_id', 'game_date', 'event_index',
        'period', 'game_seconds', 'event', 'description',
        'event_detail', 'event_zone', 'event_team', 'event_player_1',
        'event_player_2', 'event_player_3', 'event_length', 
        'number_on', 'number_off', 'jumping_on', 'jumping_off',
        'home_on_1', 'home_on_2', 'home_on_3', 'home_on_4', 'home_on_5',
        'home_on_6', 'home_on_7', 'home_on_8', 'home_on_9', 'away_on_1', 'away_on_2', 'away_on_3',
        'away_on_4', 'away_on_5', 'away_on_6', 'away_on_7', 'away_on_8', 'away_on_9', 'home_goalie',
        'away_goalie', 'home_team', 'away_team', 'home_skaters', 'away_skaters',
        'home_score', 'away_score', 'game_score_state', 'game_strength_state']

    game = game.loc[:, columns].rename(
    columns = {'period':'game_period', 'event':'event_type', 'description':'event_description', 'number_on':'num_on', 'number_off':'num_off',
              'jumping_on':'players_on', 'jumping_off':'players_off'}
    )

    return game

# FIX EVENTS WITH MISSING COORDINATES THAT CAN BE FIXED.

In [58]:
def fix_missing(single, event_coords, events):
    
    # FIRST FIX: EVENTS THAT HAVE MATCHING PERIOD, SECONDS, AND EVENT TYPE, AND ONLY OCCURRED ONCE, BUT NO EVENT PLAYER. #
    global event_coords_temp
    global single_problems
    global merged_problems
    problems = events[(events.event.isin(ewc)) & (pd.isna(events.coords_x))]
    single_problems = problems.groupby(['event', 'period', 'game_seconds'])[
        'event_index'].count().reset_index().rename(
        columns = {'event_index':'problematic_events'})
    # Keep events where only one event of that class happened at that moment.
    single_problems = single_problems[single_problems.problematic_events==1]
    single_problems = problems.merge(single_problems).drop(
        columns = ['problematic_events', 'coords_x', 'coords_y']) # x/y come back later!
    event_coords_temp = event_coords.loc[:, ['period', 'game_seconds', 'event', 'version', 'coords_x']].groupby(
    ['game_seconds', 'period', 'event', 'version'])['coords_x'].count().reset_index().rename(
        columns = {'coords_x':'problematic_events'})
    event_coords_temp = event_coords_temp[event_coords_temp.problematic_events==1].drop(columns = 'problematic_events')
    event_coords_temp = event_coords_temp.merge(event_coords.loc[:, ['game_seconds', 'period', 'event', 'version', 'coords_x', 'coords_y']])
    if 'espn_id' in event_coords_temp.columns:
        event_coords_temp = event_coords_temp.drop(columns = 'espn_id')
    merged_problems = single_problems.merge(event_coords_temp)
    print("You fixed: " + str(len(merged_problems)) + " events!")
    events = events[~(events.event_index.isin(list(merged_problems.event_index)))]
    events = pd.concat([events, merged_problems.loc[:, list(events.columns)]]).sort_values(by = ['event_index', 'period', 'game_seconds'])
    #if len(merged_problems)>0:
        #events = events[~events.event_index.isin(merged_problems.event_index)]
        #events = pd.concat([events, merged_problems.loc[:, list(events.columns)]]).sort_values(by = ['event_index', 'period', 'game_seconds'])
    look = events
    
    # SECOND FIX: EVENTS THAT HAVE MATCHING PERIOD, EVENT TYPE, AND PLAYER ONE, AND ONLY OCCURRED ONCE, BUT NO GAME SECONDS.
    
    problems = events[(events.event.isin(ewc)) & (pd.isna(events.coords_x))]
    single_problems = problems.groupby(['event', 'period', 'event_player_1'])[
        'event_index'].count().reset_index().rename(
        columns = {'event_index':'problematic_events'})
    # Keep events where only one event of that class happened at that moment.
    single_problems = single_problems[single_problems.problematic_events==1]
    single_problems = problems.merge(single_problems).drop(
        columns = ['problematic_events', 'coords_x', 'coords_y']) # x/y come back later!
    event_coords_temp = event_coords.loc[:, ['period', 'event_player_1', 'event', 
                                        'version', 'coords_x']].groupby(
    ['event_player_1', 'period', 'event', 'version'])['coords_x'].count().reset_index().rename(
        columns = {'coords_x':'problematic_events'})
    event_coords_temp = event_coords_temp[event_coords_temp.problematic_events==1].drop(columns = 'problematic_events')
    event_coords_temp = event_coords_temp.merge(event_coords.loc[:, ['event_player_1', 'period', 'event', 'version', 'coords_x', 'coords_y']])
    merged_problems = single_problems.merge(event_coords_temp)
    print("You fixed: " + str(len(merged_problems)) + " events!")
    events = events[~events.event_index.isin(merged_problems.event_index)]
    events = pd.concat([events, merged_problems]).sort_values(by = ['event_index', 'period', 'game_seconds'])
    
    return events

# FULL SCRAPE 1 BY 1 FUNCTION

In [335]:
def full_scrape_1by1(game_id_list, shift_to_espn = False):
    
    global single
    global event_coords
    global full
    global fixed_events
    global events
    
    full = pd.DataFrame()
    
    i = 0
    
    while i in range(0, len(game_id_list)):
       
        # First thing to try: Scraping HTML events
        
        try:
            first_time = time.time()
            game_id = game_id_list[i]
            print('Attemtping scrape for: ' + str(game_id))
            season = str(int(str(game_id)[:4])) + str(int(str(game_id)[:4]) + 1)
            small_id = str(game_id)[5:]
            single = scrape_html_events(season, small_id)
            single['game_id'] = int(game_id)
            
            # If all goes well with the HTML scrape:
            
            try:
                event_coords = scrape_api_events(game_id, shift_to_espn = shift_to_espn)
                api_coords = event_coords
                if len(event_coords[(event_coords.event.isin(ewc)) & (pd.isna(event_coords.coords_x))]) > 0:
                    raise ExpatError('Bad takes, dude!')
                event_coords['game_id'] = int(game_id)
                events = single.merge(event_coords, on = ['event_player_1', 'game_seconds', 'version', 'period', 'game_id', 'event'], how = 'left')
                events['coordinate_source'] = 'api'
                try:
                    events = fix_missing(single, event_coords, events)
                except IndexError as e:
                    print('Issue when fixing problematic events. Here it is: ' + str(e))
                    continue
                try:
                    shifts = scrape_html_shifts(season, small_id)
                    finalized = merge_and_prepare(events, shifts)
                    finalized['coordinate_source'] = 'api'
                    full = full.append(finalized)
                    second_time = time.time()
                except IndexError as e:
                    print('There was no shift data for this game. Error: ' + str(e))
                    fixed_events = events
                    fixed_events = fixed_events.rename(
                    columns = {'period':'game_period', 'event':'event_type', 'away_team_abbreviated':'away_team', 
                              'home_team_abbreviated':'home_team', 'description':'event_description', 'home_team':'hometeamfull',
                              'away_team':'awayteamfull'}
                    ).drop(
                    columns = ['original_time', 'other_team', 'strength', 'event_player_str', 'version', 'hometeamfull', 'awayteamfull']
                    ).assign(game_warning = 'NO SHIFT DATA.')
                    full = full.append(fixed_events)
                print('Successfully scraped ' + str(game_id) + '. Coordinates sourced from the API.')
                print("This game took " + str(round(second_time - first_time, 2)) + " seconds.")
                i = i + 1
                
                # If there is an issue with the API:
                
            except KeyError: 
                print('The API gave us trouble with: ' + str(game_id) + '. Let us try ESPN.')
                
                try:
                    home_team = single['home_team'].iloc[0]
                    away_team = single['away_team'].iloc[0]
                    game_date = single['game_date'].iloc[0]
                    try:
                        espn_id = scrape_espn_ids_single_game(str(game_date.date()), home_team, away_team).espn_id.iloc[0]
                        event_coords = scrape_espn_events(int(espn_id))
                        events = single.merge(event_coords, on = ['event_player_1', 'game_seconds', 'period', 'version', 'event'], how = 'left').drop(columns = ['espn_id'])
                        try:
                            events = fix_missing(single, event_coords, events)
                        except IndexError as e:
                            print('Issue when fixing problematic events. Here it is: ' + str(e))
                            continue
                    except IndexError:
                        print('This game does not have ESPN or API coordinates. You will get it anyway, though.')
                        events = single
                    try:
                        shifts = scrape_html_shifts(season, small_id)
                        finalized = merge_and_prepare(events, shifts)
                        finalized['coordinate_source'] = 'espn'
                        full = full.append(finalized)
                        second_time = time.time()
                    except IndexError as e:
                        print('There was no shift data for this game. Error: ' + str(e))
                        fixed_events = events
                        fixed_events = fixed_events.rename(
                        columns = {'period':'game_period', 'event':'event_type', 'away_team_abbreviated':'away_team', 
                                  'home_team_abbreviated':'home_team', 'description':'event_description', 'home_team':'hometeamfull',
                                  'away_team':'awayteamfull'}
                        ).drop(
                        columns = ['original_time', 'other_team', 'strength', 'event_player_str', 'version', 'hometeamfull', 'awayteamfull']
                        ).assign(game_warning = 'NO SHIFT DATA')
                        fixed_events['coordinate_source'] = 'espn'
                        full = full.append(fixed_events)
                    second_time = time.time()
                    # Fix this so it doesn't say sourced from ESPN if no coords.
                    if single.equals(events):
                        print("This game took " + str(round(second_time - first_time, 2)) + " seconds.")
                        i = i + 1
                    else:
                        print('Successfully scraped ' + str(game_id) + '. Coordinates sourced from ESPN.')
                        print("This game took " + str(round(second_time - first_time, 2)) + " seconds.")
                        i = i + 1
                    
                    # If there are issues with ESPN
                    
                except KeyError as e:
                    print('ESPN also had trouble scraping coordinates for: ' + str(game_id) + '. Looks like we will need to punt this one, unfortunately.')
                    print('KeyError: ' + str(e))
                    i = i + 1
                    continue
                except IndexError as e:
                    print('ESPN also had trouble scraping coordinates for: ' + str(game_id) + '. Looks like we will need to punt this one, unfortunately.')
                    print('IndexError: ' + str(e))
                    i = i + 1
                    continue
                except TypeError as e:
                    print('ESPN also had trouble scraping coordinates for: ' + str(game_id) + '. Looks like we will need to punt this one, unfortunately.')
                    print('TypeError: ' + str(e))
                    i = i + 1
                    continue
                except ExpatError as e:
                    print('ESPN also had trouble scraping coordinates for: ' + str(game_id) + '. Looks like we will need to punt this one, unfortunately.')
                    print('ExpatError: ' + str(e))
                    i = i + 1
                    continue
                
            except ExpatError:
                print('There was a rare error with the API; numerous takeaways did not have location coordinates for: ' + str(game_id) + '. Let us try ESPN.')
                
                try:
                    home_team = single['home_team'].iloc[0]
                    away_team = single['away_team'].iloc[0]
                    game_date = single['game_date'].iloc[0]
                    try:
                        espn_id = scrape_espn_ids_single_game(str(game_date.date()), home_team, away_team).espn_id.iloc[0]
                        event_coords = scrape_espn_events(int(espn_id))
                        duped_coords = api_coords.assign(source = 'api').merge(event_coords.drop(columns = 'espn_id'), on = ['game_seconds', 'event', 'period', 'version'], how = 'outer')
                        duped_coords = duped_coords.assign(source = np.where(pd.isna(duped_coords.source), 'espn', duped_coords.source))
                        duped_coords = duped_coords.assign(coords_x = np.where(pd.isna(duped_coords.coords_x_x), duped_coords.coords_x_y, duped_coords.coords_x_x),
                                                          coords_y = np.where(pd.isna(duped_coords.coords_y_x), duped_coords.coords_y_y, duped_coords.coords_y_x),
                                                          event_player_1 = duped_coords.event_player_1_x)
                        duped_coords = duped_coords.loc[:, api_coords.columns]
                        duped_coords = duped_coords[duped_coords.event.isin(['SHOT', 'HIT', 'BLOCK', 'MISS', 'GIVE', 'TAKE', 'GOAL', 'PENL', 'FAC'])]
                        duped_coords = duped_coords[~duped_coords.duplicated()]
                        event_coords = duped_coords
                        events = single.merge(event_coords, on = ['event_player_1', 'game_seconds', 'period', 'version', 'event'], how = 'left')#.drop(columns = ['espn_id'])
                        try:
                            events = fix_missing(single, event_coords, events)
                            events['coordinate_source'] = 'mix'
                        except IndexError as e:
                            print('Issue when fixing problematic events. Here it is: ' + str(e))
                    except IndexError as e:
                        if event_coords is not None:
                            print('Okay, ESPN had issues. We will go back to the API for this one. Issue: ' + str(e))
                            events = single.merge(event_coords, on = ['event_player_1', 'game_seconds', 'version', 'period', 'event'], how = 'left')
                            try:
                                events = fix_missing(single, event_coords, events)
                            except IndexError as e:
                                print('Issue when fixing problematic events. Here it is: ' + str(e))
                        else:
                            print('This game does not have ESPN or API coordinates. You will get it anyway, though. Issue: ' + str(e))
                            events = single
                            events['coordinate_source'] = 'none'
                    try:
                        shifts = scrape_html_shifts(season, small_id)
                        finalized = merge_and_prepare(events, shifts)
                        full = full.append(finalized)
                        second_time = time.time()
                    except IndexError as e:
                        print('There was no shift data for this game. Error: ' + str(e))
                        fixed_events = events
                        fixed_events = fixed_events.rename(
                        columns = {'period':'game_period', 'event':'event_type', 'away_team_abbreviated':'away_team', 
                                  'home_team_abbreviated':'home_team', 'description':'event_description', 'home_team':'hometeamfull',
                                  'away_team':'awayteamfull'}
                        ).drop(
                        columns = ['original_time', 'other_team', 'strength', 'event_player_str', 'version', 'hometeamfull', 'awayteamfull']
                        ).assign(game_warning = 'NO SHIFT DATA')
                        full = full.append(fixed_events)
                    second_time = time.time()
                    # Fix this so it doesn't say sourced from ESPN if no coords.
                    print('Successfully scraped ' + str(game_id) + '. Coordinates sourced from ESPN.')
                    print("This game took " + str(round(second_time - first_time, 2)) + " seconds.")
                    i = i + 1
                    
                    # If there are issues with ESPN
                    
                except KeyError as e:
                    print('ESPN also had trouble scraping coordinates for: ' + str(game_id) + '. Looks like we will need to punt this one, unfortunately.')
                    print('KeyError: ' + str(e))
                    i = i + 1
                    continue
                except IndexError as e:
                    print('ESPN also had trouble scraping coordinates for: ' + str(game_id) + '. Looks like we will need to punt this one, unfortunately.')
                    print('IndexError: ' + str(e))
                    i = i + 1
                    continue
                except TypeError as e:
                    print('ESPN also had trouble scraping coordinates for: ' + str(game_id) + '. Looks like we will need to punt this one, unfortunately.')
                    print('TypeError: ' + str(e))
                    i = i + 1
                    continue
                except ExpatError as e:
                    print('ESPN also had trouble scraping coordinates for: ' + str(game_id) + '. Looks like we will need to punt this one, unfortunately.')
                    print('ExpatError: ' + str(e))
                    i = i + 1
                    continue
            
        except ConnectionError:
            print('Got a Connection Error, time to sleep.')
            time.sleep(10)
            continue
            
        except AttributeError as e:
            print(str(game_id) + ' does not have an HTML report. Here is the error: ' + str(e))
            i = i + 1
            continue
            
        except IndexError as e:
            print(str(game_id) + ' has an issue with the HTML Report. Here is the error: ' + str(e))
            i = i + 1
            continue
            
        except ValueError as e:
            print(str(game_id) + ' has an issue with the HTML Report. Here is the error: ' + str(e))
            i = i + 1
            continue
            
        except KeyboardInterrupt:
            print('You manually interrupted the scrape. You will get to keep every game you have already completed scraping. Good bye.')
            global hidden_patrick
            hidden_patrick = 1
            return full
        
    return full

# USER-END FUNCTION: full_scrape

# PROVIDE A LIST TO full_scrape, YOU WILL RECEIVE A DATAFRAME THAT INCLUDES EVERY GAME FROM YOUR LIST THAT WAS SUCCESSFULLY SCRAPED.

In [338]:
def full_scrape(game_id_list, shift = False):
    
    hidden_patrick = 0
    
    df = full_scrape_1by1(game_id_list, shift_to_espn = shift)
    return df
    
    if hidden_patrick==0:
        
        gids = list(set(fix0910.game_id))
        missing = [x for x in game_id_list if x not in gids]
        print('You missed the following games: ' + str(missing))
        print('Let us try scraping each of them one more time.')
        retry = full_scrape_1by1(missing)
        if retry is not None:
            df = df.append(retry)
            return df
        else:
            return df
    
    else:
        return df

# EXAMPLE RUN: 

In [345]:
# Note this list ends at 2011020005

target_games = list(range(2011020001, 2011020006))

In [346]:
example_scrape_1011 = full_scrape(target_games)

Attemtping scrape for: 2011020001
You fixed: 0 events!
You fixed: 0 events!
Successfully scraped 2011020001. Coordinates sourced from the API.
This game took 4.96 seconds.
Attemtping scrape for: 2011020002
You fixed: 0 events!
You fixed: 0 events!
Successfully scraped 2011020002. Coordinates sourced from the API.
This game took 5.06 seconds.
Attemtping scrape for: 2011020003
You fixed: 0 events!
You fixed: 0 events!
Successfully scraped 2011020003. Coordinates sourced from the API.
This game took 5.55 seconds.
Attemtping scrape for: 2011020004
You fixed: 0 events!
You fixed: 0 events!
Successfully scraped 2011020004. Coordinates sourced from the API.
This game took 5.32 seconds.
Attemtping scrape for: 2011020005
You fixed: 0 events!
You fixed: 0 events!
Successfully scraped 2011020005. Coordinates sourced from the API.
This game took 6.07 seconds.


# Findings from comparing single games:

- 09-10: ESPN Coords x are 1 higher, coords y 1 lower, except opening faceoff with x 1 higher and y 2 lower, also a play with exact same coords.
- 10-11: ESPN Coords x are 1 higher, coords y 1 lower, except opening faceoff with coords x and coords y both 2 higher.
- 11-12: ESPN Coords x are 1 higher, coords y 1 lower, except opening faceoff with coords x 2 higher and coords y 2 lower.
- 2013: ESPN x coords are 1 higher and coords y are 1 lower.
- 13-14: ESPN Coords x are 1 higher, coords y 1 lower, except opening faceoff with coords x 1 higher and coords y the same.
- 14-15: ESPN Coords x are 1 higher, coords y 1 lower, except opening faceoff with coords x 2 higher and coords y the same.
- 15-16: ESPN Coords x are 1 higher, coords y are 1 lower.